# Livestock number projections under ssps

In [ ]:
%%capture
!pip install netcdf4
import os, fnmatch, numpy as np, pandas as pd, xarray as xr
import pylab as plt, plotly.express as px, seaborn as sns
from matplotlib.gridspec import GridSpec; from matplotlib.colors import from_levels_and_colors as flc
import cv2; from google.colab.patches import cv2_imshow
from scipy.interpolate import make_interp_spline, BSpline
%load_ext google.colab.data_table 

#!pip install zarr gcsfs cftime nc-time-axis 
#import gcsfs, zarr; gcs = gcsfs.GCSFileSystem( token = 'anon' )
from google.colab import drive; drive.mount( '/content/drive' )

plt.rcParams.update( {'font.size': 8, 'xtick.labelsize' : 6, 'ytick.labelsize' : 6, 'legend.fontsize': 6, 'legend.frameon': False, 
                     'axes.linewidth':0.4, 'xtick.major.width':0.4, 'ytick.major.width':0.4, 'xtick.major.size':1, 'ytick.major.size':1,
                     'xtick.bottom': True, 'xtick.top': False, 'ytick.right': False, 'ytick.right': False, 'legend.numpoints': 4,
                     'xtick.major.pad':2.5, 'ytick.major.pad':2.5, 'axes.labelpad':2,'xtick.direction': 'out', 'ytick.direction': 'out', 
                      'axes.edgecolor':'black','ytick.color':'black','xtick.color':'black','axes.facecolor':'white'} )

plt.savefig('carly.png', dpi = 500, bbox_inches = 'tight', pad_inches = 0.01 )

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fexperimentsandconfigs%20https%3a%2f%2fwww.googleapis.com%2fauth%2fphotos.native&response_type=code

Enter your authorization code:
4/1AX4XfWjmsVoctn2mErAUMy38hfMfzShRrhffcmAmprNIDVsFnWMWTPTcAe8


In [ ]:
def rgb2hex( rgbstring ): 
    s = rgbstring.split( '(' )[1].split( ',' )
    return "#{0:02x}{1:02x}{2:02x}".format(max(0,min(np.int(s[0]),255)),max(0,min(np.int(s[1]),255)),max(0,min(np.int(s[2].split(')')[0]),255)))

### Array of grid areas ###
nlon = 1440
lat = np.linspace( -89.875, 89.875, 720) * np.pi / 180 # in radians
area = ( 2 * np.pi * 6.3781e3 ) / ( ( lat.shape[0] + 1 ) * 2 ) * ( 2 * np.pi *  6.3781e3 * np.cos(lat) ) / ( nlon + 1 )
cellarea = np.repeat( area[:,np.newaxis], nlon, axis = 1 )

def canonical2continent(country):
    if country in ["Algeria", "Angola", "Benin", "Botswana", "Burkina", "Burundi", "Cameroon", "Cape Verde", "Central African Republic", 
                   "Chad", "Comoros", "Congo", "Congo, Democratic Republic of", "Djibouti", "Egypt", "Equatorial Guinea", "Eritrea", 
                   "Ethiopia", "Gabon", "Gambia", "Ghana", "Guinea", "Guinea-Bissau", "Ivory Coast", "Kenya", "Lesotho", "Liberia", 
                   "Libya", "Madagascar", "Malawi", "Mali", "Mauritania", "Mauritius", "Morocco", "Mozambique", "Namibia", "Niger", 
                   "Nigeria", "Rwanda", "Sao Tome and Principe", "Senegal", "Seychelles", "Sierra Leone", "Somalia", "South Africa", 
                   "South Sudan", "Sudan", "Swaziland", "Tanzania", "Togo", "Tunisia", "Uganda", "Zambia", "Zimbabwe", "Canary Isl. (Spain)", 
                   "Congo, R. of", "Cote d'Ivoire", "France (Mayotte)", "France (Réunion)", "France (Mozambique Channel Isl.)"]:
        return "Africa"

    elif country in ["Afghanistan", "Bahrain", "Bangladesh", "Bhutan", "Brunei", "Burma (Myanmar)", "Cambodia", "China", "East Timor", "India", 
                     "Indonesia", "Iran", "Iraq", "Israel", "Japan", "Jordan", "Kazakhstan", "Korea, North", "Korea, South", "Kuwait", 
                     "Kyrgyzstan", "Laos", "Lebanon", "Malaysia", "Maldives", "Mongolia", "Nepal", "Oman", "Pakistan", "Philippines", "Qatar", 
                     "Russian Federation", "Saudi Arabia", "Singapore", "Sri Lanka", "Syria", "Tajikistan", "Thailand", "Turkey", "Turkmenistan", 
                     "United Arab Emirates", "Uzbekistan", "Vietnam", "Yemen", "Hong Kong", "Malaysia (Peninsula East)", "Korea (South)", 
                     "Turkey (Black Sea)", "Saudi Arabia (Persian Gulf)", "Myanmar", "Taiwan", "Russia", "Andaman & Nicobar Isl. (India)", "Korea (North)"]:
        return "Asia"

    elif country in ["Albania", "Andorra", "Armenia", "Austria", "Azerbaijan", "Belarus", "Belgium", "Bosnia and Herzegovina", "Bulgaria", "Croatia", 
                     "Cyprus", "Czech Republic", "Denmark", "Estonia", "Finland", "France", "Georgia", "Germany", "Greece", "Hungary", "Iceland", 
                     "Ireland", "Italy", "Latvia", "Liechtenstein", "Lithuania", "Luxembourg", "Macedonia", "Malta", "Moldova", "Monaco", "Montenegro", 
                     "Netherlands", "Norway", "Poland", "Portugal", "Romania", "San Marino", "Serbia", "Slovakia", "Slovenia", "Spain", "Sweden", 
                     "Switzerland", "Ukraine", "United Kingdom", "Vatican City", "Norway (Svalbard Isl.)", "Channel Isl. (UK)"]:
        return "Europe"

    elif country in ["Antigua and Barbuda", "Bahamas", "Barbados", "Belize", "Canada", "Costa Rica", "Cuba", "Dominica", "Dominican Republic", "El Salvador", 
                     "Grenada", "Guatemala", "Haiti", "Honduras", "Jamaica", "Mexico", "Nicaragua", "Panama", "Saint Kitts and Nevis", "Saint Lucia", 
                     "Saint Vincent and the Grenadines", "Trinidad and Tobago", "United States", "Alaska (USA)", "Greenland", "Haiti (Navassa Isl.)", 
                     "Puerto Rico (USA)", "France (Martinique)", "Turks & Caicos Isl. (UK)", "US Virgin Isl.", "France (Guadeloupe)", "Antigua & Barbuda", 
                     "British Virgin Isl. (UK)", "Anguilla (UK)", "Saint Vincent & the Grenadines", "Bermuda (UK)", "Cayman Isl. (UK)", "Montserrat (UK)"]:
        return "North America"

    elif country in ["Australia", "Fiji", "Kiribati", "Marshall Islands", "Micronesia", "Nauru", "New Zealand", "Palau", "Papua New Guinea", "Samoa", 
                     "Solomon Islands", "Tonga", "Tuvalu", "Vanuatu", "American Samoa", "Australia (Heard & McDonald Isl.)", "Hawaii", 
                     "Australia (Macquarie Isl.)", "Kermadec Isl. (New Zealand)", "Australia (Lord Howe Isl.)", "Marshall Isl.", "France (New Caledonia)", 
                     "Solomon Isl.", "Palmyra Atoll & Kingman Reef (USA)", "Brunei Darussalam", "Guam (USA)", "Howland & Baker Isl. (USA)", 
                     "France (French Polynesia)", "Australia (Christmas Isl.)", "Johnston Atoll (USA)", "Northern Marianas (USA)", "Cook Isl. (New Zealand)", 
                     "Timor Leste", "Australia (Norfolk Isl.)", "France (Wallis & Futuna Isl.)"]:
        return "Oceania"

    elif country in ["Argentina", "Bolivia", "Bolivia (Plurinational State of)", "Brazil", "Chile", "Colombia", "Ecuador", "Guyana", "Paraguay", "Peru", "Suriname", "Uruguay", "Venezuela", 
                     "Falkland Isl. (UK)", "France (French Guiana)", "Netherlands Antilles (Windward)", "Trinidad & Tobago", "Chile (Desventuradas Isl.)", 
                     "Brazil (Trindade & Martin Vaz Isl.)"]:
        return "South America"

    elif country in ["Chagos Archipel., Brit. Ind. Oc. Terr. (UK)", "Azores Isl. (Portugal)", "Chile (J. Fernandez, Felix and Ambrosio Isl.)", 
                     "Ecuador (Galapagos Isl.)", "Amsterdam & St Paul Isl. (France)", "Madeira Isl. (Portugal)", "Jarvis Isl. (USA)", 
                     "Australia (Cocos (Keeling) Isl.)", "Tristan da Cunha Isl. (UK)", "Chile (Easter Isl.)", "South Georgia & Sandwich Isl. (UK)", 
                     "Denmark (Faeroe Isl.)", "Jan Mayen Isl. (Norway)", "France (Kerguelen Isl.)"]:
        return "Open Ocean"
    else: return None

# 1. SSP livestock number increase at the regional level

In [ ]:
### SSP projections from IIASA ###
iam = pd.read_csv('/content/drive/My Drive/data/econ/ssps/SSP_IAM_V2_201811.csv')
iam = iam[iam.VARIABLE == 'Agricultural Demand|Livestock']
iam = iam.melt( id_vars = ['MODEL', 'SCENARIO', 'REGION', 'VARIABLE', 'UNIT'], var_name = 'year', value_name = 'value' )
iam = iam.groupby( by = ['SCENARIO', 'year', 'REGION'], as_index = False ).mean()
iam.year = pd.to_numeric( iam.year )

### get percent change from 2010 baseline ###
iam2010 = iam[ iam.year == 2010 ]
iam2010 = iam2010.drop( columns = ('year') ).rename( columns = {'value':'baseline'} )
iam = pd.merge( iam, iam2010, on = ['SCENARIO', 'REGION'] )
iam['change'] = (iam.value - iam.baseline) / iam.baseline * 100
iam['ssp']  = iam.SCENARIO.apply( lambda x: x.split('-')[0] )
iam = iam.drop( columns = ['value', 'baseline'] )

### region designations from FAO ###
regions = pd.read_excel('/content/drive/My Drive/data/econ/ssps/cmip6_iam_model_region_mapping.xlsx')
regions = regions[ ['ISO', 'Country', 'R5_region'] ]
regions= regions.replace( ['R5LAM', 'R5MAF', 'R5REF', 'R5OECD', 'R5ASIA'], ['R5.2LAM', 'R5.2MAF', 'R5.2REF', 'R5.2OECD', 'R5.2ASIA'] )
regions = regions.replace({'United States':'United States of America',
                           'Bolivia': 'Bolivia (Plurinational State of)',
                           'Libyan Arab Jamahiriya': 'Libya',
                           'Venezuela':'Venezuela (Bolivarian Republic of)',
                           'Russia':'Russian Federation',
                           'Cote dIvoire': "Côte d'Ivoire",
                           'Laos': "Lao People's Democratic Republic",
                           'Burma':'Myanmar',
                           'Korea, Democratic Peoples Republic of':"Democratic People's Republic of Korea",
                           'Korea,  Republic of': 'Republic of Korea'})

### livestock numbers from FAO ###
df = pd.read_csv( '/content/drive/My Drive/data/livestock/country_level_data/Environment_LivestockPatterns_E_All_Data_NOFLAG.csv', sep = ',', encoding = 'iso-8859-1' )
df = df.drop( columns = ['Area Code', 'Element Code', 'Item Code'] )
df = df.melt( id_vars = ['Area', 'Element', 'Item', 'Unit'], var_name = 'year', value_name = 'value' ).rename( columns = { 'Area' : 'Country' } )
df.year = df.year.apply( lambda x: x.split('Y')[1] ).astype( int )
df = df[df.Element == 'Stocks']
#ruminants = ['Goat','Sheep','Cattle','Buffaloes']
ruminants = ['Cattle']
df = df[df.Item.isin(ruminants)]
df = df[df.Country != 'World']
df = pd.merge( df, regions, on = ['Country'] ).rename( columns = {'R5_region':'REGION'})
df = df[ df.year==2010 ]
df = df.drop( columns = ('year') )
df = df.groupby( ['REGION'], as_index = False ).sum().rename(columns = {'value':'cattle2010'})
df.cattle2010 = df.cattle2010 / 1e6

### Combine country level livestock data with regional ssp projected changes ###
df = pd.merge(df, iam, on = ['REGION'])
df['cattle'] = df.cattle2010 * (1 + df.change / 100)
df['addcattle'] = (df.cattle - df.cattle2010)

df = df[df['SCENARIO'].str.contains('Baseline')]
df = df[df.year>2005]

df.head(3)

In [ ]:
### plot the futures of cows ##
def cow_futures_plot():
    regions = ['R5.2OECD','R5.2ASIA', 'R5.2LAM', 'R5.2MAF']
    colors = [rgb2hex(i) for i in px.colors.sequential.amp[2::2]]
    colors = px.colors.qualitative.Plotly

    plt.rcParams.update( {'font.size': 8, 'xtick.labelsize' : 6, 'ytick.labelsize' : 6, 'legend.fontsize': 6, 'legend.frameon': False, 
                        'axes.linewidth':0.4, 'xtick.major.width':0.4, 'ytick.major.width':0.4, 'xtick.major.size':1, 'ytick.major.size':1,
                        'xtick.bottom': True, 'xtick.top': False, 'ytick.right': False, 'ytick.right': False, 'legend.numpoints': 4,
                        'xtick.major.pad':3, 'ytick.major.pad':3, 'axes.labelpad':3,'xtick.direction': 'out', 'ytick.direction': 'out', 
                        'axes.edgecolor':'black','ytick.color':'black','xtick.color':'black','axes.facecolor':'white'} )

    plt.close('all')
    fig = plt.figure( figsize = (7, 7 ))
    gs = GridSpec(2, 2, height_ratios = [1,1])#, width_ratios = [0.4, 1, 0.05] )
    gs.update(wspace = 0.05, hspace = 0.05)
    ssps = ['SSP1-Baseline', 'SSP2-Baseline', 'SSP3-Baseline', 'SSP4-Baseline','SSP5-Baseline']

    ax1 = plt.subplot(gs[0,0])
    dft = df[df.REGION == regions[0]]
    plt.annotate(regions[0], xy = (2015,800))
    for i, ssp in enumerate(ssps):
        dft1 = dft[dft.SCENARIO == ssp]
        plt.scatter(dft1.year, dft1.cattle, facecolors = 'none', edgecolors=colors[i], lw =0.8, alpha = 0.4, marker = 's', zorder = 2)
        plt.plot(dft1.year, dft1.cattle, color= colors[i], zorder = 0, label = ssp)

    plt.ylim(100,900)
    plt.yticks([100,200,300,400,500,600,700,800,900],[100,200,300,400,500,600,700,800,900])
    plt.xticks([2020,2040,2060,2080,2100],[])
    sns.despine( offset = 3, trim = True )
    plt.yticks( rotation = 90, va = 'center' )
    plt.ylabel('Million Cattle')
    plt.legend()

    ax1 = plt.subplot(gs[0,1])
    dft = df[df.REGION == regions[1]]
    plt.annotate(regions[1], xy = (2015,800))
    for i, ssp in enumerate(ssps):
        dft1 = dft[dft.SCENARIO == ssp]
        plt.scatter(dft1.year, dft1.cattle, facecolors = 'none', edgecolors=colors[i], lw =0.8, alpha = 0.4, marker = 's', zorder = 2)
        plt.plot(dft1.year, dft1.cattle, color= colors[i], zorder = 0)

    plt.ylim(100,900)
    plt.yticks([100,200,300,400,500,600,700,800,900],[])
    plt.xticks([2020,2040,2060,2080,2100],[])
    sns.despine( offset = 3, trim = True )
    plt.yticks( rotation = 90, va = 'center' )

    ax1 = plt.subplot(gs[1,0])
    dft = df[df.REGION == regions[2]]
    plt.annotate(regions[2], xy = (2015,800))
    for i, ssp in enumerate(ssps):
        dft1 = dft[dft.SCENARIO == ssp]
        plt.scatter(dft1.year, dft1.cattle, facecolors = 'none', edgecolors=colors[i], lw =0.8, alpha = 0.4, marker = 's', zorder = 2)
        plt.plot(dft1.year, dft1.cattle, color= colors[i], zorder = 0)

    plt.ylim(100,900)
    plt.yticks([100,200,300,400,500,600,700,800,900],[100,200,300,400,500,600,700,800,900])
    plt.xticks([2020,2040,2060,2080,2100],[2020,2040,2060,2080,2100])
    sns.despine( offset = 3, trim = True )
    plt.yticks( rotation = 90, va = 'center' )
    plt.ylabel('Million Cattle')
    plt.xlabel('Year')

    ax1 = plt.subplot(gs[1,1])
    dft = df[df.REGION == regions[3]]
    plt.annotate(regions[3], xy = (2015,800))
    for i, ssp in enumerate(ssps):
        dft1 = dft[dft.SCENARIO == ssp]
        plt.scatter(dft1.year, dft1.cattle, facecolors = 'none', edgecolors=colors[i], lw =0.8, alpha = 0.4, marker = 's', zorder = 2)
        plt.plot(dft1.year, dft1.cattle, color= colors[i], zorder = 0)

    plt.ylim(100,900)
    plt.yticks([100,200,300,400,500,600,700,800,900],[])
    plt.xticks([2020,2040,2060,2080,2100],[2020,2040,2060,2080,2100])
    sns.despine( offset = 3, trim = True )
    plt.yticks( rotation = 90, va = 'center' )
    plt.xlabel('Year')


    plt.savefig( f'/content/drive/My Drive/figures/livestock/supplemental_region_cows.png', dpi = 500, bbox_inches = 'tight', pad_inches = 0.00 )

cow_futures_plot()

# 2. Land Use Change at the country level --> merge with regional livestock projections


In [ ]:
### pick an ssp ###
ssp = 370      ###
### pick an ssp ###

### get unique list of countries from FAO and GADM countries... need to rename some GADM countries to FAO designation 
dfc = pd.read_csv( '/content/drive/My Drive/data/livestock/country_level_data/Production_Livestock_E_All_Data_NOFLAG.csv', sep = ',', encoding = 'iso-8859-1' )
dfc = dfc[['Area','Item']].rename( columns = {'Area':'countryname'} )
dfc = dfc[dfc.Item == 'Cattle']
GADM =  pd.read_csv( '/content/drive/My Drive/data/area_masks/gadm36_country_names_and_abbr.csv', sep = ',' )
GADM = GADM.rename( columns = {'NAME_0':'countryname', 'GID_0': 'ISO'} )
GADM['maskid'] = GADM.index
GADM = GADM[['ISO','maskid']]
FAO =  pd.read_csv( '/content/drive/My Drive/data/area_masks/countrycodes_FAO_NAMES.csv', sep = ',' ).rename(columns = {'abb':'ISO'})
FAO = FAO[['ISO','countryname']]
countrynames = pd.merge( FAO, GADM, on = [ 'ISO' ] , how = 'inner' )
smallisland = ['Barbados','Bermuda' ,'Dominica','Bahama','Grenada','Guam','Montserrat','Malta','Nauru','Niue','Saint Kitts and Nevis', 
               'Saint Pierre and Miquelon', 'St. Vincent and the Grenadines','Singapore','Tonga','Tuvalu','Wallis and Futuna Islands']
dfc = dfc[ ~dfc.countryname.isin(smallisland) ]
dfc = pd.merge( dfc, countrynames, on = [ 'countryname' ] , how = 'inner' )
dfc = dfc.rename( columns = {'maskid': 'country'} ).drop( columns = ['Item'] )
######----------------------------------------------------------------------------------------------#####

### Land use data from LUH2 ###
dfla = pd.read_csv( f'/content/drive/My Drive/data/livestock/land_use/ssp{ssp}_country_level.csv' ) ###ssp370
dfla = pd.merge( dfla, dfc, on = 'country' ).drop(columns = ['countryname','country'])
dfla = pd.merge(dfla, regions, on = 'ISO').rename( columns = {'R5_region':'REGION'})
dfla['land'] = dfla.pastr + dfla.rang + dfla.crop

dflabase = dfla[dfla.year == 2015]
dflabase = dflabase[['ISO','land']].rename(columns ={ 'land': 'land2015'})
dfla = pd.merge( dfla, dflabase, on = 'ISO' )
dfla['landchange'] = dfla.land - dfla.land2015 

dflaO= dfla[dfla.REGION == 'R5.2OECD']
dflaregO = dflaO.groupby(['REGION','year'], as_index=False)['landchange'].sum().rename(columns ={ 'landchange': 'region_landchange'})
dflaO = pd.merge( dflaO, dflaregO, on = ['REGION', 'year'] )
dflaO['contribution'] = dflaO.landchange / dflaO.region_landchange

dfla = dfla[dfla.REGION != 'R5.2OECD']
dflareg = dfla[ dfla.landchange > 0 ] ### only take positive land change areas ###
dflareg = dflareg.groupby(['REGION','year'], as_index=False)['landchange'].sum().rename(columns ={ 'landchange': 'region_landchange'})
dfla = pd.merge( dfla, dflareg, on = ['REGION', 'year'] )
dfla['contribution'] = dfla.landchange / dfla.region_landchange

dfla = pd.concat( (dfla, dflaO) )

decades = [2015, 2025, 2035, 2045, 2055, 2065, 2075, 2085, 2095]
dfnew = pd.DataFrame({})
for year in decades:
    dft = dfla[dfla.year>year]
    dft = dft[dft.year<year+10]
    dft = dft.groupby( ['ISO','Country','REGION'], as_index = False ).mean()
    dfnew = pd.concat( (dfnew, dft) )

dfnew = dfnew.replace(2097.5, 2100)
dfnew.year = dfnew.year.astype(int)
dfnew.head(3)

### combine regional animal gains with country-level land change
df = df[ df.SCENARIO == 'SSP3-Baseline' ] ######## CHANNNNNNGEEEEEE MEEEEEEEEEEE #####
df = pd.merge( dfnew, df, on = ['REGION','year'] )
df['newcows'] = df.addcattle * df.contribution

### livestock numbers from FAO to get country level in 2010 again ###
dfa = pd.read_csv( '/content/drive/My Drive/data/livestock/country_level_data/Environment_LivestockPatterns_E_All_Data_NOFLAG.csv', sep = ',', encoding = 'iso-8859-1' )
dfa = dfa.drop( columns = ['Area Code', 'Element Code', 'Item Code'] )
dfa = dfa.melt( id_vars = ['Area', 'Element', 'Item', 'Unit'], var_name = 'year', value_name = 'value' ).rename( columns = { 'Area' : 'Country' } )
dfa.year = dfa.year.apply( lambda x: x.split('Y')[1] ).astype( int )
dfa = dfa[dfa.Element == 'Stocks']
dfa = dfa[dfa.Item.isin(['Cattle'])]
dfa = dfa[dfa.Country != 'World']
dfa = dfa[ dfa.year==2010 ]
dfa = pd.merge( dfa, regions, on = ['Country'] )
dfa = dfa[['ISO', 'value']].rename(columns={'value':'cows_country_2010'})
dfa.cows_country_2010 = dfa.cows_country_2010 / 1e6

df = pd.merge(df, dfa, on = ['ISO'])

### remove negative projections ###
dfO = df[df.REGION == 'R5.2OECD']
df = df[df.REGION != 'R5.2OECD']
df.newcows[df.newcows < 0 ] = 0
df = pd.concat(( df, dfO ))
df['projected_cows'] = (df.newcows + df.cows_country_2010)

df.head(3)

In [ ]:
### plot the futures of cows in each country in a specific SSP ##
def cow_futures_plot():
    regions = ['R5.2OECD','R5.2ASIA', 'R5.2LAM', 'R5.2MAF']
    colors = [rgb2hex(i) for i in px.colors.sequential.deep[1::3]]

    plt.rcParams.update( {'font.size': 8, 'xtick.labelsize' : 6, 'ytick.labelsize' : 6, 'legend.fontsize': 6, 'legend.frameon': False, 
                        'axes.linewidth':0.4, 'xtick.major.width':0.4, 'ytick.major.width':0.4, 'xtick.major.size':1, 'ytick.major.size':1,
                        'xtick.bottom': True, 'xtick.top': False, 'ytick.right': False, 'ytick.right': False, 'legend.numpoints': 4,
                        'xtick.major.pad':3, 'ytick.major.pad':3, 'axes.labelpad':3,'xtick.direction': 'out', 'ytick.direction': 'out', 
                        'axes.edgecolor':'black','ytick.color':'black','xtick.color':'black','axes.facecolor':'white'} )

    plt.close('all')
    fig = plt.figure( figsize = (7, 7 ))
    gs = GridSpec(2, 2, height_ratios = [1,1])#, width_ratios = [0.4, 1, 0.05] )
    gs.update(wspace = 0.05, hspace = 0.05)
    ssps = ['SSP1-Baseline', 'SSP2-Baseline', 'SSP3-Baseline', 'SSP4-Baseline','SSP5-Baseline']

    ax1 = plt.subplot(gs[0,0])
    dft = df[df.REGION == regions[0]]
    plt.annotate(regions[0], xy = (2020,300), color = colors[0])
    countries = dft.Country.unique()
    for country in countries:
        dft1 = dft[dft.Country == country ]
        plt.plot(dft1.year, dft1.projected_cows, color= colors[0], zorder = 0, lw = 0.8)
        if dft1.projected_cows.values[-1] > 20: 
            plt.annotate( dft1.ISO.unique()[0], xy = (2101, dft1.projected_cows.values[-1]-2), fontsize = 5, color = colors[0], zorder = 10 )

    plt.xlim(2020,2105)
    plt.ylim(0,310)
    plt.yticks([0,100,200,300],[0,100,200,300])
    plt.xticks([2020,2040,2060,2080,2100],[])
    sns.despine( offset = 3, trim = True )
    plt.yticks( rotation = 90, va = 'center' )
    plt.ylabel('Million Cattle')

    ax1 = plt.subplot(gs[0,1])
    dft = df[df.REGION == regions[1]]
    plt.annotate(regions[1], xy = (2020,300), color = colors[1])
    countries = dft.Country.unique()
    for country in countries:
        dft1 = dft[dft.Country == country ]
        plt.plot(dft1.year, dft1.projected_cows, color= colors[1], zorder = 0, lw = 0.8)
        if dft1.projected_cows.values[-1] > 20: 
            plt.annotate( dft1.ISO.unique()[0], xy = (2101, dft1.projected_cows.values[-1]-2), fontsize = 5, color = colors[1], zorder = 10 )

    plt.xlim(2020,2105)
    plt.ylim(0,310)
    plt.yticks([0,100,200,300],[])
    plt.xticks([2020,2040,2060,2080,2100],[])
    sns.despine( offset = 3, trim = True )
    plt.yticks( rotation = 90, va = 'center' )

    ax1 = plt.subplot(gs[1,0])
    dft = df[df.REGION == regions[2]]
    plt.annotate(regions[2], xy = (2020,290), color = colors[2])
    countries = dft.Country.unique()
    for country in countries:
        dft1 = dft[dft.Country == country ]
        plt.plot(dft1.year, dft1.projected_cows, color= colors[2], zorder = 0, lw = 0.8)
        if dft1.projected_cows.values[-1] > 20: 
            plt.annotate( dft1.ISO.unique()[0], xy = (2101, dft1.projected_cows.values[-1]-2), fontsize = 5, color = colors[2], zorder = 10 )

    plt.xlim(2020,2105)
    plt.ylim(0,310)
    plt.yticks([0,100,200,300],[0,100,200,300])
    plt.xticks([2020,2040,2060,2080,2100],[2020,2040,2060,2080,2100])
    sns.despine( offset = 3, trim = True )
    plt.yticks( rotation = 90, va = 'center' )
    plt.ylabel('Million Cattle')
    plt.xlabel('Year')

    ax1 = plt.subplot(gs[1,1])
    dft = df[df.REGION == regions[3]]
    plt.annotate(regions[3], xy = (2020,290), color = colors[3])
    countries = dft.Country.unique()
    for country in countries:
        dft1 = dft[dft.Country == country ]
        plt.plot(dft1.year, dft1.projected_cows, color= colors[3], zorder = 0, lw = 0.8)
        if dft1.projected_cows.values[-1] > 20: 
            plt.annotate( dft1.ISO.unique()[0], xy = (2101, dft1.projected_cows.values[-1]-2), fontsize = 5, color = colors[3], zorder = 10 )

    plt.xlim(2020,2105)
    plt.ylim(0,310)
    plt.yticks([0,100,200,300],[])
    plt.xticks([2020,2040,2060,2080,2100],[2020,2040,2060,2080,2100])
    sns.despine( offset = 3, trim = True )
    plt.yticks( rotation = 90, va = 'center' )
    plt.xlabel('Year')

    plt.savefig( f'/content/drive/My Drive/figures/livestock/supplemental_region_cows_370_by_country.png', dpi = 500, bbox_inches = 'tight', pad_inches = 0.00 )

cow_futures_plot()

# 3. Climate projections (df calculated elsewhere)

In [ ]:
### load historical data ###
root = '/content/drive/My Drive/data/livestock/cmip6_country/'
files = os.listdir(root)
files = fnmatch.filter(files, "*hist*HL*68*")
files = [root + i for i in files]
dft = pd.DataFrame()
for file in files:
    df1 = pd.read_csv(file)
    df1['model'] = file.split('/')[-1].split('_')[0]
    dft = pd.concat((dft,df1))

dfh = dft.groupby( ['country','model'], as_index = False ).mean().drop( columns = ['year'] ).rename(columns = {'produ':'pro_h'})

files = os.listdir(root)
files = fnmatch.filter(files, f"*{ssp}*HL*dynamic*68*")
files = [root + i for i in files]
dft = pd.DataFrame()
for file in files:
    df1 = pd.read_csv(file)
    df1['model'] = file.split('/')[-1].split('_')[0]
    dft = pd.concat( (dft, df1) )
#df['ssp'] = 'SSP5-85'

dft = pd.merge( dft, dfh,  on = ['country','model'], how = 'left' )
dft = pd.merge( dft, dfc, on =['country'], how = 'left').drop(columns =['country'] )
dft.year = dft.year + 5

### annoying issue with 2095-2100 not being the same as the other decades ###
#df2090 = dft[dft.year != 2100]
#df2100 = dft[dft.year == 2100]
#df2100b = df2100[df2100.model == 'MRI-ESM2-0']
#df2100 = df2100[df2100.model != 'MRI-ESM2-0']
#df2100.produ *= 1.67 # six years... except for MRI, which is the full decade

#dft = pd.concat( ( df2090, df2100 ) )
dft['heat_increase'] = dft.produ - dft.pro_h
dft = dft.dropna()
dft = dft[['year', 'ISO','heat_increase','model']]

### merge with cow data ###
df = pd.merge(df, dft, on = ['year','ISO'])

dft.head(3)

In [ ]:
df = get_full_ssp(370)
df.model.unique()

In [ ]:
df[df.Country =='China']


# Figure 4 panel B

In [ ]:
dft = df[['year', 'ISO','Country','REGION','cattle','newcows','heat_increase','model','cows_country_2010', 'projected_cows']]
dft = dft.groupby(['year', 'ISO','Country','REGION'], as_index= False ).mean()
dft.heat_increase = dft.heat_increase / 3650 ## decadal to daily
#dft = dft[dft.year == 2070]

regions = ['R5.2OECD','R5.2ASIA', 'R5.2LAM', 'R5.2MAF']
colors = [rgb2hex(i) for i in px.colors.sequential.deep[2::3]]

plt.rcParams.update( {'font.size': 8, 'xtick.labelsize' : 6, 'ytick.labelsize' : 6, 'legend.fontsize': 6, 'legend.frameon': False, 
                     'axes.linewidth':0.4, 'xtick.major.width':0.4, 'ytick.major.width':0.4, 'xtick.major.size':1, 'ytick.major.size':1,
                     'xtick.bottom': True, 'xtick.top': False, 'ytick.right': False, 'ytick.right': False, 'legend.numpoints': 4,
                     'xtick.major.pad':3, 'ytick.major.pad':3, 'axes.labelpad':3,'xtick.direction': 'out', 'ytick.direction': 'out', 
                      'axes.edgecolor':'black','ytick.color':'black','xtick.color':'black','axes.facecolor':'white'} )

plt.close('all')
fig = plt.figure( figsize = (5, 5 ), frameon=True)

size_factor = 20
for i, region in enumerate(regions):
    dft1 = dft[dft.REGION == region]
    for index, row in dft1.iterrows():
        if row.projected_cows > 30: 
            plt.scatter( row.heat_increase, row.newcows, facecolors = colors[i], edgecolors='black', lw =0.8, alpha = 0.4, s = row.projected_cows * size_factor)
            plt.annotate( row.ISO, xy = (row.heat_increase-0.15, row.newcows- 1), fontsize = 5, zorder = 100 )
        else: 
            plt.scatter( row.heat_increase, row.newcows, facecolors = colors[i], edgecolors='none', lw =0.8, alpha = 0.4, s = row.projected_cows * size_factor)

plt.scatter(8.5, -11, s = 250 * size_factor, facecolors = 'gray', edgecolors='gray', lw =0.2, alpha = 0.2 )
plt.scatter(8.5, -22, s = 100 * size_factor, facecolors = 'gray', edgecolors='gray', lw =0.2, alpha = 0.2 )
plt.scatter(8.5, -35, s = 10  * size_factor, facecolors = 'gray', edgecolors='gray', lw =0.2, alpha = 0.2 )

plt.ylim( -42.5, 192) 
plt.xlim( -0.1, 9.5 ) 
plt.yticks( [-40,-20,0,20,40,60,80,100,120,140,160,180], [-40,-20,0,20,40,60,80,100,120,140,160,180] )
plt.xticks( [0,1,2,3,4,5,6,7,8,9], [0,1,2,3,4,5,6,7,8,9] )
plt.axhline(0, lw = 0.3, xmax = 0.75, linestyle = '--', color = 'gray')
sns.despine( offset = 3, trim = True )
plt.yticks( rotation = 90, va = 'center' )

plt.savefig( '/content/drive/My Drive/figures/livestock/figure_4_2090_68.png', dpi = 500, bbox_inches = 'tight', pad_inches = 0.01 )

# Figure 4, other panels


In [ ]:
### pick an ssp  to get full data ###
def get_full_ssp(ssp):
    ### SSP projections from IIASA ###
    iam = pd.read_csv('/content/drive/My Drive/data/econ/ssps/SSP_IAM_V2_201811.csv')
    iam = iam[iam.VARIABLE == 'Agricultural Demand|Livestock']
    iam = iam.melt( id_vars = ['MODEL', 'SCENARIO', 'REGION', 'VARIABLE', 'UNIT'], var_name = 'year', value_name = 'value' )
    iam = iam.groupby( by = ['SCENARIO', 'year', 'REGION'], as_index = False ).mean()
    iam.year = pd.to_numeric( iam.year )

    ### get percent change from 2010 baseline ###
    iam2010 = iam[ iam.year == 2010 ]
    iam2010 = iam2010.drop( columns = ('year') ).rename( columns = {'value':'baseline'} )
    iam = pd.merge( iam, iam2010, on = ['SCENARIO', 'REGION'] )
    iam['change'] = (iam.value - iam.baseline) / iam.baseline * 100
    iam['ssp']  = iam.SCENARIO.apply( lambda x: x.split('-')[0] )
    iam = iam.drop( columns = ['value', 'baseline'] )

    ### region designations from FAO ###
    regions = pd.read_excel('/content/drive/My Drive/data/econ/ssps/cmip6_iam_model_region_mapping.xlsx')
    regions = regions[ ['ISO', 'Country', 'R5_region'] ]
    regions= regions.replace( ['R5LAM', 'R5MAF', 'R5REF', 'R5OECD', 'R5ASIA'], ['R5.2LAM', 'R5.2MAF', 'R5.2REF', 'R5.2OECD', 'R5.2ASIA'] )
    regions = regions.replace({'United States':'United States of America',
                            'Bolivia': 'Bolivia (Plurinational State of)',
                            'Libyan Arab Jamahiriya': 'Libya',
                            'Venezuela':'Venezuela (Bolivarian Republic of)',
                            'Russia':'Russian Federation',
                            'Cote dIvoire': "Côte d'Ivoire",
                            'Laos': "Lao People's Democratic Republic",
                            'Burma':'Myanmar',
                            'Korea, Democratic Peoples Republic of':"Democratic People's Republic of Korea",
                            'Korea,  Republic of': 'Republic of Korea'})

    ### livestock numbers from FAO ###
    df = pd.read_csv( '/content/drive/My Drive/data/livestock/country_level_data/Environment_LivestockPatterns_E_All_Data_NOFLAG.csv', sep = ',', encoding = 'iso-8859-1' )
    df = df.drop( columns = ['Area Code', 'Element Code', 'Item Code'] )
    df = df.melt( id_vars = ['Area', 'Element', 'Item', 'Unit'], var_name = 'year', value_name = 'value' ).rename( columns = { 'Area' : 'Country' } )
    df.year = df.year.apply( lambda x: x.split('Y')[1] ).astype( int )
    df = df[df.Element == 'Stocks']
    #ruminants = ['Goat','Sheep','Cattle','Buffaloes']
    ruminants = ['Cattle']
    df = df[df.Item.isin(ruminants)]
    df = df[df.Country != 'World']
    df = pd.merge( df, regions, on = ['Country'] ).rename( columns = {'R5_region':'REGION'})
    df = df[ df.year==2010 ]
    df = df.drop( columns = ('year') )
    df = df.groupby( ['REGION'], as_index = False ).sum().rename(columns = {'value':'cattle2010'})
    df.cattle2010 = df.cattle2010 / 1e6

    ### Combine country level livestock data with regional ssp projected changes ###
    df = pd.merge(df, iam, on = ['REGION'])
    df['cattle'] = df.cattle2010 * (1 + df.change / 100)
    df['addcattle'] = (df.cattle - df.cattle2010)

    df = df[df['SCENARIO'].str.contains('Baseline')]
    df = df[df.year>2005]

    ### get unique list of countries from FAO and GADM countries... need to rename some GADM countries to FAO designation 
    dfc = pd.read_csv( '/content/drive/My Drive/data/livestock/country_level_data/Production_Livestock_E_All_Data_NOFLAG.csv', sep = ',', encoding = 'iso-8859-1' )
    dfc = dfc[['Area','Item']].rename( columns = {'Area':'countryname'} )
    dfc = dfc[dfc.Item == 'Cattle']
    GADM =  pd.read_csv( '/content/drive/My Drive/data/area_masks/gadm36_country_names_and_abbr.csv', sep = ',' )
    GADM = GADM.rename( columns = {'NAME_0':'countryname', 'GID_0': 'ISO'} )
    GADM['maskid'] = GADM.index
    GADM = GADM[['ISO','maskid']]
    FAO =  pd.read_csv( '/content/drive/My Drive/data/area_masks/countrycodes_FAO_NAMES.csv', sep = ',' ).rename(columns = {'abb':'ISO'})
    FAO = FAO[['ISO','countryname']]
    countrynames = pd.merge( FAO, GADM, on = [ 'ISO' ] , how = 'inner' )
    smallisland = ['Barbados','Bermuda' ,'Dominica','Bahama','Grenada','Guam','Montserrat','Malta','Nauru','Niue','Saint Kitts and Nevis', 
                'Saint Pierre and Miquelon', 'St. Vincent and the Grenadines','Singapore','Tonga','Tuvalu','Wallis and Futuna Islands']
    dfc = dfc[ ~dfc.countryname.isin(smallisland) ]
    dfc = pd.merge( dfc, countrynames, on = [ 'countryname' ] , how = 'inner' )
    dfc = dfc.rename( columns = {'maskid': 'country'} ).drop( columns = ['Item'] )
    ######----------------------------------------------------------------------------------------------#####

    ### Land use data from LUH2 ###
    dfla = pd.read_csv( f'/content/drive/My Drive/data/livestock/land_use/ssp{ssp}_country_level.csv' ) ###ssp370
    dfla = pd.merge( dfla, dfc, on = 'country' ).drop(columns = ['countryname','country'])
    dfla = pd.merge(dfla, regions, on = 'ISO').rename( columns = {'R5_region':'REGION'})
    dfla['land'] = dfla.pastr + dfla.rang + dfla.crop

    dflabase = dfla[dfla.year == 2015]
    dflabase = dflabase[['ISO','land']].rename(columns ={ 'land': 'land2015'})
    dfla = pd.merge( dfla, dflabase, on = 'ISO' )
    dfla['landchange'] = dfla.land - dfla.land2015 

    dflaO= dfla[dfla.REGION == 'R5.2OECD']
    dflaregO = dflaO.groupby(['REGION','year'], as_index=False)['landchange'].sum().rename(columns ={ 'landchange': 'region_landchange'})
    dflaO = pd.merge( dflaO, dflaregO, on = ['REGION', 'year'] )
    dflaO['contribution'] = dflaO.landchange / dflaO.region_landchange

    dfla = dfla[dfla.REGION != 'R5.2OECD']
    dflareg = dfla[ dfla.landchange > 0 ] ### only take positive land change areas ###
    dflareg = dflareg.groupby(['REGION','year'], as_index=False)['landchange'].sum().rename(columns ={ 'landchange': 'region_landchange'})
    dfla = pd.merge( dfla, dflareg, on = ['REGION', 'year'] )
    dfla['contribution'] = dfla.landchange / dfla.region_landchange

    dfla = pd.concat( (dfla, dflaO) )

    decades = [2015, 2025, 2035, 2045, 2055, 2065, 2075, 2085, 2095]
    dfnew = pd.DataFrame({})
    for year in decades:
        dft = dfla[dfla.year>year]
        dft = dft[dft.year<year+10]
        dft = dft.groupby( ['ISO','Country','REGION'], as_index = False ).mean()
        dfnew = pd.concat( (dfnew, dft) )

    dfnew = dfnew.replace(2097.5, 2100)
    dfnew.year = dfnew.year.astype(int)
    dfnew.head(3)

    ### combine regional animal gains with country-level land change
    df = df[ df.SCENARIO == f'SSP{str(ssp)[0]}-Baseline' ] 
    df = pd.merge( dfnew, df, on = ['REGION','year'] )
    df['newcows'] = df.addcattle * df.contribution

    ### livestock numbers from FAO to get country level in 2010 again ###
    dfa = pd.read_csv( '/content/drive/My Drive/data/livestock/country_level_data/Environment_LivestockPatterns_E_All_Data_NOFLAG.csv', sep = ',', encoding = 'iso-8859-1' )
    dfa = dfa.drop( columns = ['Area Code', 'Element Code', 'Item Code'] )
    dfa = dfa.melt( id_vars = ['Area', 'Element', 'Item', 'Unit'], var_name = 'year', value_name = 'value' ).rename( columns = { 'Area' : 'Country' } )
    dfa.year = dfa.year.apply( lambda x: x.split('Y')[1] ).astype( int )
    dfa = dfa[dfa.Element == 'Stocks']
    dfa = dfa[dfa.Item.isin(['Cattle'])]
    dfa = dfa[dfa.Country != 'World']
    dfa = dfa[ dfa.year==2010 ]
    dfa = pd.merge( dfa, regions, on = ['Country'] )
    dfa = dfa[['ISO', 'value']].rename(columns={'value':'cows_country_2010'})
    dfa.cows_country_2010 = dfa.cows_country_2010 / 1e6

    df = pd.merge(df, dfa, on = ['ISO'])

    ### remove negative projections ###
    dfO = df[df.REGION == 'R5.2OECD']
    df = df[df.REGION != 'R5.2OECD']
    df.newcows[df.newcows < 0 ] = 0
    df = pd.concat(( df, dfO ))
    df['projected_cows'] = (df.newcows + df.cows_country_2010)

    ### load historical data ###
    root = '/content/drive/My Drive/data/livestock/cmip6_country/'
    files = os.listdir(root)
    files = fnmatch.filter(files, "*hist*HL*68*")
    files = [root + i for i in files]
    dft = pd.DataFrame()
    for file in files:
        df1 = pd.read_csv(file)
        df1['model'] = file.split('/')[-1].split('_')[0]
        dft = pd.concat((dft,df1))

    dfh = dft.groupby( ['country','model'], as_index = False ).mean().drop( columns = ['year'] ).rename(columns = {'produ':'pro_h'})

    files = os.listdir(root)
    files = fnmatch.filter(files, f"*{ssp}*HL*")
    files = [root + i for i in files]
    dft = pd.DataFrame()
    for file in files:
        df1 = pd.read_csv(file)
        df1['model'] = file.split('/')[-1].split('_')[0]
        dft = pd.concat( (dft, df1) )
    #df['ssp'] = 'SSP5-85'

    dft = pd.merge( dft, dfh,  on = ['country','model'], how = 'left' )
    dft = pd.merge( dft, dfc, on =['country'], how = 'left').drop(columns =['country'] )
    dft.year = dft.year + 5

    ### annoying issue with 2095-2100 not being the same as the other decades ###
    dft = dft[dft.model != 'MRI-ESM2-0']
    df2090 = dft[dft.year != 2100]
    df2100 = dft[dft.year == 2100]
    df2100.produ *= 1.67
    dft = pd.concat( ( df2090, df2100 ) )

    dft['heat_increase'] = dft.produ - dft.pro_h
    dft = dft.dropna()
    dft = dft[['year', 'ISO','heat_increase','model','produ','pro_h']]

    ### merge with cow data ###
    df = pd.merge(df, dft, on = ['year','ISO'])
    return df


In [ ]:
df = get_full_ssp(126)
dft = df[['year', 'ISO','Country','REGION','cattle','newcows','heat_increase','model','cows_country_2010', 'projected_cows','produ','pro_h']]
dft['risk'] = dft.projected_cows * dft.produ
dft['risk2010'] = dft.cows_country_2010 * dft.pro_h
df370 = dft.copy()
df370 = df370.groupby(['model','REGION','year'], as_index= False ).sum()
df370 = df370.groupby(['REGION','year'], as_index= False ).median()
df370[df370.year == 2100]

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

,REGION,year,cattle,newcows,heat_increase,cows_country_2010,projected_cows,produ,pro_h,risk,risk2010
8,R5.2ASIA,2100,15004.735175,97.970699,245855.532974,446.430061,544.400760,1.301609e+06,1.105720e+06,1.131872e+07,6.490570e+06
17,R5.2LAM,2100,18723.712024,23.384459,335544.833051,561.667462,585.051921,1.498659e+06,1.367397e+06,1.152168e+07,7.815675e+06
26,R5.2MAF,2100,69840.348145,800.284685,552312.574450,274.167870,1074.452555,2.481787e+06,2.232869e+06,1.257575e+07,3.270458e+06
35,R5.2OECD,2100,17101.001220,-33.739045,90727.196595,461.562691,427.823647,2.249281e+05,1.494761e+05,1.779492e+06,9.687411e+05


In [ ]:
print( (12964197.95613987 - 3245285.2133002817) / 3245285.2133002817 )
print( (19952660.66568434 - 3907837.625715242) /  3907837.625715242 )
print( (24646118.649333037 - 1635229.142801565) / 1635229.142801565 )

2.994779227110203
4.1058059665497115
14.071966371090932


In [ ]:
print( (11318718.955521362 - 6490570.4266005615) / 6490570.4266005615 )
print( (11521684.511728143 - 7815675.251430484) /  7815675.251430484 )
print( (12575751.002891345 - 3270458.2856031284) / 3270458.2856031284 )

0.7438712180262937
0.4741764647423597
2.8452565067871403


In [ ]:
df = get_full_ssp(370)
dft = df[['year', 'ISO','Country','REGION','cattle','newcows','heat_increase','model','cows_country_2010', 'projected_cows']]
dft.heat_increase = dft.heat_increase / 3650 ## decadal to daily
dft['risk'] = (dft.projected_cows * dft.heat_increase) / 1000
dft['risk2010'] = (dft.cows_country_2010 * dft.heat_increase) / 1000
df370 = dft.copy()
df370 = df370.groupby(['model','REGION','year'], as_index= False ).sum()
df370 = df370.groupby(['REGION','year'], as_index= False ).median()
df370[df370.year == 2100]

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas

,REGION,year,cattle,newcows,heat_increase,cows_country_2010,projected_cows,risk,risk2010
8,R5.2ASIA,2100,11781.789239,229.456605,104.119367,223.215030,452.671635,1.931946,0.945923
17,R5.2LAM,2100,19898.575202,340.964704,143.769253,280.833731,621.798435,3.653592,1.396232
26,R5.2MAF,2100,54252.128307,697.370355,273.186458,137.083935,784.137359,3.884390,0.608973
35,R5.2OECD,2100,7515.932404,-43.842871,42.126573,230.781346,186.938474,0.339531,0.409963
40,R5.2REF,2100,568.841929,15.193811,12.771152,32.209683,47.403494,0.042693,0.034768


In [ ]:
print( (1.931945985913377 - 0.9459226586116007) / 0.9459226586116007 )
print( (3.65359238190701 - 1.3962322215609329) / 1.3962322215609329 )
print( (3.8843896037221652 - 0.6089729410982447) / 0.6089729410982447 )

1.0423931791094148
1.6167512291203514
5.3785914637141525


In [ ]:
print( (1.3357216618723013-0.6569791531116315) / 0.3002371480822395 )
print( (3.1007009695289955- 1.1212390139885482) / 0.5106441806468952 )
print( (3.400741822224661-0.5398422011126123) / 0.1835537629098421 )

2.260687969813622
3.8764016717723515
15.586167103080664


In [ ]:
df = get_full_ssp(126)
dft = df[['year', 'ISO','Country','REGION','cattle','newcows','heat_increase','model','cows_country_2010', 'projected_cows']]
dft.heat_increase = dft.heat_increase / 3650 ## decadal to daily
dft['risk'] = (dft.projected_cows * dft.heat_increase) / 1000
dft['risk2010'] = (dft.cows_country_2010 * dft.heat_increase) / 1000
df126 = dft.copy()
df126 = df126.groupby(['model','REGION','year'], as_index= False ).sum()
df126 = df126.groupby(['REGION','year'], as_index= False ).median()
df126[df126.year == 2100]

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas

,REGION,year,cattle,newcows,heat_increase,cows_country_2010,projected_cows,risk,risk2010
8,R5.2ASIA,2100,15004.735175,97.970699,67.357680,446.430061,544.400760,0.970759,0.779625
17,R5.2LAM,2100,18723.712024,23.384459,91.930091,561.667462,585.051921,1.067410,1.038735
26,R5.2MAF,2100,69840.348145,800.284685,151.318514,274.167870,1074.452555,0.964075,0.321815
35,R5.2OECD,2100,17101.001220,-33.739045,24.856766,461.562691,427.823647,0.231643,0.252034


In [ ]:
print( (1.931945985913377 - 0.9459226586116007) / 0.9459226586116007 )
print( (3.65359238190701 - 1.3962322215609329) / 1.3962322215609329 )
print( (3.8843896037221652 - 0.6089729410982447) / 0.6089729410982447 )

In [ ]:
colors = [rgb2hex(i) for i in px.colors.sequential.deep[2::3]]

plt.close('all')
fig = plt.figure( figsize = (10, 4.5 ))
gs = GridSpec( 2, 4, height_ratios = [2,0.75] ) 
gs.update(wspace = 0.05, hspace = 0.05)

newyears = np.arange( 2020, 2101, 1)
k = 2

def make_subplot( region, colorid, ylim, ticks, xticks, yaxis=True):
    dft = df370[df370.REGION == region]
    models = dft.model.unique()
    for model in models:
        dft1 = dft[dft.model == model]
        spl1 = make_interp_spline(dft1.year.values, dft1.risk.values, k=k)
        plt.plot( newyears, spl1(newyears),  lw = 0.4, color = colors[colorid], zorder = 3)

    years = [2020,2030,2040,2050,2060,2070,2080,2090,2100]
    riskmax = []
    riskmin = []
    riskmed = []
    for year in years:
        dft1 = dft[dft.year == year]
        riskmax.append( dft1.risk.values.max() )
        riskmin.append( dft1.risk.values.min() )
        riskmed.append( dft1.risk.values.mean())

    spl1 = make_interp_spline( years, riskmin, k=k )
    spl2 = make_interp_spline( years, riskmax, k=k )
    plt.fill_between(newyears, y1 = spl1(newyears),   y2 = spl2(newyears), facecolor = colors[colorid], edgecolor = 'none', alpha = 0.2, zorder = 2 )
    spl1 = make_interp_spline( years, riskmed, k=k )
    plt.plot(newyears, spl1(newyears),   lw = 1.4, color = 'black', zorder = 5)

    dft = df126[df126.REGION == region]
    models = dft.model.unique()
    for model in models:
        dft1 = dft[dft.model == model]
        spl1 = make_interp_spline(dft1.year.values, dft1.risk.values, k=k)
        plt.plot( newyears, spl1(newyears),  lw = 0.4, color = '#bdbdbd', zorder = 1)

    years = [2020,2030,2040,2050,2060,2070,2080,2090,2100]
    riskmax = []
    riskmin = []
    riskmed = []
    for year in years:
        dft1 = dft[dft.year == year]
        riskmax.append( dft1.risk.values.max() )
        riskmin.append( dft1.risk.values.min() )
        riskmed.append( dft1.risk.values.mean())

    spl1 = make_interp_spline( years, riskmin, k=k )
    spl2 = make_interp_spline( years, riskmax, k=k )
    plt.fill_between(newyears, y1 = spl1(newyears),   y2 = spl2(newyears), facecolor = '#bdbdbd', edgecolor = 'none', alpha = 0.4, zorder = 0 )
    spl1 = make_interp_spline( years, riskmed, k=k )
    plt.plot(newyears, spl1(newyears),   lw = 1.4, color = 'black', zorder = 2)

    plt.xticks([2020,2040,2060,2080,2100], xticks)
    plt.ylim(0,ylim)
    plt.yticks(ticks, ticks)

    if yaxis == False: 
        ax1.axes.get_yaxis().set_visible(False)
        plt.yticks([1], [])
        sns.despine( offset = 3, trim = True )
    else:
        sns.despine( offset = 3, trim = True )

ax1 = plt.subplot(gs[0,0])
make_subplot( 'R5.2OECD', 0, 6.2, [0,1,2,3,4,5,6],[])

ax1 = plt.subplot(gs[0,1])
make_subplot( 'R5.2ASIA', 1, 6.2, [],[], False )

ax1 = plt.subplot(gs[0,2])
make_subplot( 'R5.2LAM', 2, 6.2, [],[], False )

ax1 = plt.subplot(gs[0,3])
make_subplot( 'R5.2MAF', 3, 6.2, [],[], False )

ax1 = plt.subplot(gs[1,0])
dft = df370[df370.REGION == 'R5.2OECD']
years = [2020,2030,2040,2050,2060,2070,2080,2090,2100]
riskmed = []
riskmed2010 = []
for year in years:
    dft1 = dft[dft.year == year]
    riskmed.append( dft1.risk.values.mean() )
    riskmed2010.append( dft1.risk2010.values.mean() )

spl1 = make_interp_spline( years, riskmed, k=k )
spl2 = make_interp_spline( years, riskmed2010, k=k )
plt.fill_between(newyears, y1 = 0,  y2 = spl2(newyears), facecolor = colors[0], edgecolor = 'none', alpha = 0.2, zorder = 2 )
plt.fill_between(newyears, y1 = spl2(newyears),  y2 = spl1(newyears), facecolor = colors[0], edgecolor = 'none', alpha = 0.5, zorder = 2 )
plt.plot(newyears, spl1(newyears), lw = 1.4, color = 'black', zorder = 2)

plt.ylim(0,4.5)
plt.xticks([2020,2040,2060,2080,2100], [2020,2040,2060,2080,2100])
ax1.axes.get_yaxis().set_visible(False)
plt.yticks([1], [])
sns.despine( offset = 3, trim = True )

ax1 = plt.subplot(gs[1,1])
dft = df370[df370.REGION == 'R5.2ASIA']
years = [2020,2030,2040,2050,2060,2070,2080,2090,2100]
riskmed = []
riskmed2010 = []
for year in years:
    dft1 = dft[dft.year == year]
    riskmed.append( dft1.risk.values.mean() )
    riskmed2010.append( dft1.risk2010.values.mean() )

spl1 = make_interp_spline( years, riskmed, k=k )
spl2 = make_interp_spline( years, riskmed2010, k=k )
plt.fill_between(newyears, y1 = 0,  y2 = spl2(newyears), facecolor = colors[1], edgecolor = 'none', alpha = 0.2, zorder = 2 )
plt.fill_between(newyears, y1 = spl2(newyears),  y2 = spl1(newyears), facecolor = colors[1], edgecolor = 'none', alpha = 0.5, zorder = 2 )
plt.plot(newyears, spl1(newyears), lw = 1.4, color = 'black', zorder = 2)

plt.ylim(0,4.5)
plt.xticks([2020,2040,2060,2080,2100], [2020,2040,2060,2080,2100])
ax1.axes.get_yaxis().set_visible(False)
plt.yticks([1], [])
sns.despine( offset = 3, trim = True )

ax1 = plt.subplot(gs[1,2])
dft = df370[df370.REGION == 'R5.2LAM']
years = [2020,2030,2040,2050,2060,2070,2080,2090,2100]
riskmed = []
riskmed2010 = []
for year in years:
    dft1 = dft[dft.year == year]
    riskmed.append( dft1.risk.values.mean() )
    riskmed2010.append( dft1.risk2010.values.mean() )

spl1 = make_interp_spline( years, riskmed, k=k )
spl2 = make_interp_spline( years, riskmed2010, k=k )
plt.fill_between(newyears, y1 = 0,  y2 = spl2(newyears), facecolor = colors[2], edgecolor = 'none', alpha = 0.2, zorder = 2 )
plt.fill_between(newyears, y1 = spl2(newyears),  y2 = spl1(newyears), facecolor = colors[2], edgecolor = 'none', alpha = 0.5, zorder = 2 )
plt.plot(newyears, spl1(newyears), lw = 1.4, color = 'black', zorder = 2)

plt.ylim(0,4.5)
plt.xticks([2020,2040,2060,2080,2100], [2020,2040,2060,2080,2100])
ax1.axes.get_yaxis().set_visible(False)
plt.yticks([1], [])
sns.despine( offset = 3, trim = True )

ax1 = plt.subplot(gs[1,3])
dft = df370[df370.REGION == 'R5.2MAF']
years = [2020,2030,2040,2050,2060,2070,2080,2090,2100]
riskmed = []
riskmed2010 = []
for year in years:
    dft1 = dft[dft.year == year]
    riskmed.append( dft1.risk.values.mean() )
    riskmed2010.append( dft1.risk2010.values.mean() )

spl1 = make_interp_spline( years, riskmed, k=k )
spl2 = make_interp_spline( years, riskmed2010, k=k )
plt.fill_between(newyears, y1 = 0,  y2 = spl2(newyears), facecolor = colors[3], edgecolor = 'none', alpha = 0.2, zorder = 2 )
plt.fill_between(newyears, y1 = spl2(newyears),  y2 = spl1(newyears), facecolor = colors[3], edgecolor = 'none', alpha = 0.5, zorder = 2 )
plt.plot(newyears, spl1(newyears), lw = 1.4, color = 'black', zorder = 2)

plt.ylim(0,4.5)
plt.xticks([2020,2040,2060,2080,2100], [2020,2040,2060,2080,2100])
ax1.axes.get_yaxis().set_visible(False)
plt.yticks([1], [])
sns.despine( offset = 3, trim = True )

plt.savefig( '/content/drive/My Drive/figures/livestock/figure_4_main.png', dpi = 900, bbox_inches = 'tight', pad_inches = 0.00 )

In [ ]:
colors = [rgb2hex(i) for i in px.colors.sequential.deep[2::3]]

plt.close('all')
fig = plt.figure( figsize = (10, 4.5 ))
gs = GridSpec( 2, 4, height_ratios = [2,0.75] ) 
gs.update(wspace = 0.05, hspace = 0.05)

newyears = np.arange( 2020, 2101, 1)
k = 2

def make_subplot( region, colorid, ylim, ticks, xticks, yaxis=True):
    dft = df370[df370.REGION == region]
    models = dft.model.unique()
    for model in models:
        dft1 = dft[dft.model == model]
        spl1 = make_interp_spline(dft1.year.values, dft1.risk.values, k=k)
        plt.plot( newyears, spl1(newyears),  lw = 0.4, color = colors[colorid], zorder = 3)

    years = [2020,2030,2040,2050,2060,2070,2080,2090,2100]
    riskmax = []
    riskmin = []
    riskmed = []
    for year in years:
        dft1 = dft[dft.year == year]
        riskmax.append( dft1.risk.values.max() )
        riskmin.append( dft1.risk.values.min() )
        riskmed.append( dft1.risk.values.mean())

    spl1 = make_interp_spline( years, riskmin, k=k )
    spl2 = make_interp_spline( years, riskmax, k=k )
    plt.fill_between(newyears, y1 = spl1(newyears),   y2 = spl2(newyears), facecolor = colors[colorid], edgecolor = 'none', alpha = 0.2, zorder = 2 )
    spl1 = make_interp_spline( years, riskmed, k=k )
    plt.plot(newyears, spl1(newyears),   lw = 1.4, color = 'black', zorder = 5)

    dft = df126[df126.REGION == region]
    models = dft.model.unique()
    for model in models:
        dft1 = dft[dft.model == model]
        spl1 = make_interp_spline(dft1.year.values, dft1.risk.values, k=k)
        plt.plot( newyears, spl1(newyears),  lw = 0.4, color = '#bdbdbd', zorder = 1)

    years = [2020,2030,2040,2050,2060,2070,2080,2090,2100]
    riskmax = []
    riskmin = []
    riskmed = []
    for year in years:
        dft1 = dft[dft.year == year]
        riskmax.append( dft1.risk.values.max() )
        riskmin.append( dft1.risk.values.min() )
        riskmed.append( dft1.risk.values.mean())

    spl1 = make_interp_spline( years, riskmin, k=k )
    spl2 = make_interp_spline( years, riskmax, k=k )
    plt.fill_between(newyears, y1 = spl1(newyears),   y2 = spl2(newyears), facecolor = '#bdbdbd', edgecolor = 'none', alpha = 0.4, zorder = 0 )
    spl1 = make_interp_spline( years, riskmed, k=k )
    plt.plot(newyears, spl1(newyears),   lw = 1.4, color = 'black', zorder = 2)

    plt.xticks([2020,2040,2060,2080,2100], xticks)
    plt.ylim(0,ylim)
    plt.yticks(ticks, ticks)

    if yaxis == False: 
        ax1.axes.get_yaxis().set_visible(False)
        plt.yticks([1], [])
        sns.despine( offset = 3, trim = True )
    else:
        sns.despine( offset = 3, trim = True )

ax1 = plt.subplot(gs[0,0])
make_subplot( 'R5.2OECD', 0, 6.2, [0,1,2,3,4,5,6],[])

ax1 = plt.subplot(gs[0,1])
make_subplot( 'R5.2ASIA', 1, 6.2, [],[], False )

ax1 = plt.subplot(gs[0,2])
make_subplot( 'R5.2LAM', 2, 6.2, [],[], False )

ax1 = plt.subplot(gs[0,3])
make_subplot( 'R5.2MAF', 3, 6.2, [],[], False )

ax1 = plt.subplot(gs[1,0])
dft = df370[df370.REGION == 'R5.2OECD']
years = [2020,2030,2040,2050,2060,2070,2080,2090,2100]
riskmed = []
riskmed2010 = []
for year in years:
    dft1 = dft[dft.year == year]
    riskmed.append( dft1.risk.values.mean() )
    riskmed2010.append( dft1.risk2010.values.mean() )

spl1 = make_interp_spline( years, riskmed, k=k )
spl2 = make_interp_spline( years, riskmed2010, k=k )
plt.fill_between(newyears, y1 = 0,  y2 = spl2(newyears), facecolor = colors[0], edgecolor = 'none', alpha = 0.2, zorder = 2 )
plt.fill_between(newyears, y1 = spl2(newyears),  y2 = spl1(newyears), facecolor = colors[0], edgecolor = 'none', alpha = 0.5, zorder = 2 )
plt.plot(newyears, spl1(newyears), lw = 1.4, color = 'black', zorder = 2)

plt.ylim(0,4.5)
plt.xticks([2020,2040,2060,2080,2100], [2020,2040,2060,2080,2100])
ax1.axes.get_yaxis().set_visible(False)
plt.yticks([1], [])
sns.despine( offset = 3, trim = True )

ax1 = plt.subplot(gs[1,1])
dft = df370[df370.REGION == 'R5.2ASIA']
years = [2020,2030,2040,2050,2060,2070,2080,2090,2100]
riskmed = []
riskmed2010 = []
for year in years:
    dft1 = dft[dft.year == year]
    riskmed.append( dft1.risk.values.mean() )
    riskmed2010.append( dft1.risk2010.values.mean() )

spl1 = make_interp_spline( years, riskmed, k=k )
spl2 = make_interp_spline( years, riskmed2010, k=k )
plt.fill_between(newyears, y1 = 0,  y2 = spl2(newyears), facecolor = colors[1], edgecolor = 'none', alpha = 0.2, zorder = 2 )
plt.fill_between(newyears, y1 = spl2(newyears),  y2 = spl1(newyears), facecolor = colors[1], edgecolor = 'none', alpha = 0.5, zorder = 2 )
plt.plot(newyears, spl1(newyears), lw = 1.4, color = 'black', zorder = 2)

plt.ylim(0,4.5)
plt.xticks([2020,2040,2060,2080,2100], [2020,2040,2060,2080,2100])
ax1.axes.get_yaxis().set_visible(False)
plt.yticks([1], [])
sns.despine( offset = 3, trim = True )

ax1 = plt.subplot(gs[1,2])
dft = df370[df370.REGION == 'R5.2LAM']
years = [2020,2030,2040,2050,2060,2070,2080,2090,2100]
riskmed = []
riskmed2010 = []
for year in years:
    dft1 = dft[dft.year == year]
    riskmed.append( dft1.risk.values.mean() )
    riskmed2010.append( dft1.risk2010.values.mean() )

spl1 = make_interp_spline( years, riskmed, k=k )
spl2 = make_interp_spline( years, riskmed2010, k=k )
plt.fill_between(newyears, y1 = 0,  y2 = spl2(newyears), facecolor = colors[2], edgecolor = 'none', alpha = 0.2, zorder = 2 )
plt.fill_between(newyears, y1 = spl2(newyears),  y2 = spl1(newyears), facecolor = colors[2], edgecolor = 'none', alpha = 0.5, zorder = 2 )
plt.plot(newyears, spl1(newyears), lw = 1.4, color = 'black', zorder = 2)

plt.ylim(0,4.5)
plt.xticks([2020,2040,2060,2080,2100], [2020,2040,2060,2080,2100])
ax1.axes.get_yaxis().set_visible(False)
plt.yticks([1], [])
sns.despine( offset = 3, trim = True )

ax1 = plt.subplot(gs[1,3])
dft = df370[df370.REGION == 'R5.2MAF']
years = [2020,2030,2040,2050,2060,2070,2080,2090,2100]
riskmed = []
riskmed2010 = []
for year in years:
    dft1 = dft[dft.year == year]
    riskmed.append( dft1.risk.values.mean() )
    riskmed2010.append( dft1.risk2010.values.mean() )

spl1 = make_interp_spline( years, riskmed, k=k )
spl2 = make_interp_spline( years, riskmed2010, k=k )
plt.fill_between(newyears, y1 = 0,  y2 = spl2(newyears), facecolor = colors[3], edgecolor = 'none', alpha = 0.2, zorder = 2 )
plt.fill_between(newyears, y1 = spl2(newyears),  y2 = spl1(newyears), facecolor = colors[3], edgecolor = 'none', alpha = 0.5, zorder = 2 )
plt.plot(newyears, spl1(newyears), lw = 1.4, color = 'black', zorder = 2)

plt.ylim(0,4.5)
plt.xticks([2020,2040,2060,2080,2100], [2020,2040,2060,2080,2100])
ax1.axes.get_yaxis().set_visible(False)
plt.yticks([1], [])
sns.despine( offset = 3, trim = True )

plt.savefig( '/content/drive/My Drive/figures/livestock/figure_4_main.png', dpi = 900, bbox_inches = 'tight', pad_inches = 0.00 )

# Figure 4 with percent change

In [ ]:
### pick an ssp  to get full data ###
def get_full_ssp(ssp):
    ### SSP projections from IIASA ###
    iam = pd.read_csv('/content/drive/My Drive/data/econ/ssps/SSP_IAM_V2_201811.csv')
    iam = iam[iam.VARIABLE == 'Agricultural Demand|Livestock']
    iam = iam.melt( id_vars = ['MODEL', 'SCENARIO', 'REGION', 'VARIABLE', 'UNIT'], var_name = 'year', value_name = 'value' )
    iam = iam.groupby( by = ['SCENARIO', 'year', 'REGION'], as_index = False ).mean()
    iam.year = pd.to_numeric( iam.year )

    ### get percent change from 2010 baseline ###
    iam2010 = iam[ iam.year == 2010 ]
    iam2010 = iam2010.drop( columns = ('year') ).rename( columns = {'value':'baseline'} )
    iam = pd.merge( iam, iam2010, on = ['SCENARIO', 'REGION'] )
    iam['change'] = (iam.value - iam.baseline) / iam.baseline * 100
    iam['ssp']  = iam.SCENARIO.apply( lambda x: x.split('-')[0] )
    iam = iam.drop( columns = ['value', 'baseline'] )

    ### region designations from FAO ###
    regions = pd.read_excel('/content/drive/My Drive/data/econ/ssps/cmip6_iam_model_region_mapping.xlsx')
    regions = regions[ ['ISO', 'Country', 'R5_region'] ]
    regions= regions.replace( ['R5LAM', 'R5MAF', 'R5REF', 'R5OECD', 'R5ASIA'], ['R5.2LAM', 'R5.2MAF', 'R5.2REF', 'R5.2OECD', 'R5.2ASIA'] )
    regions = regions.replace({'United States':'United States of America',
                            'Bolivia': 'Bolivia (Plurinational State of)',
                            'Libyan Arab Jamahiriya': 'Libya',
                            'Venezuela':'Venezuela (Bolivarian Republic of)',
                            'Russia':'Russian Federation',
                            'Cote dIvoire': "Côte d'Ivoire",
                            'Laos': "Lao People's Democratic Republic",
                            'Burma':'Myanmar',
                            'Korea, Democratic Peoples Republic of':"Democratic People's Republic of Korea",
                            'Korea,  Republic of': 'Republic of Korea'})

    ### livestock numbers from FAO ###
    df = pd.read_csv( '/content/drive/My Drive/data/livestock/country_level_data/Environment_LivestockPatterns_E_All_Data_NOFLAG.csv', sep = ',', encoding = 'iso-8859-1' )
    df = df.drop( columns = ['Area Code', 'Element Code', 'Item Code'] )
    df = df.melt( id_vars = ['Area', 'Element', 'Item', 'Unit'], var_name = 'year', value_name = 'value' ).rename( columns = { 'Area' : 'Country' } )
    df.year = df.year.apply( lambda x: x.split('Y')[1] ).astype( int )
    df = df[df.Element == 'Stocks']
    #ruminants = ['Goat','Sheep','Cattle','Buffaloes']
    ruminants = ['Cattle']
    df = df[df.Item.isin(ruminants)]
    df = df[df.Country != 'World']
    df = pd.merge( df, regions, on = ['Country'] ).rename( columns = {'R5_region':'REGION'})
    df = df[ df.year==2010 ]
    df = df.drop( columns = ('year') )
    df = df.groupby( ['REGION'], as_index = False ).sum().rename(columns = {'value':'cattle2010'})
    df.cattle2010 = df.cattle2010 / 1e6

    ### Combine country level livestock data with regional ssp projected changes ###
    df = pd.merge(df, iam, on = ['REGION'])
    df['cattle'] = df.cattle2010 * (1 + df.change / 100)
    df['addcattle'] = (df.cattle - df.cattle2010)

    df = df[df['SCENARIO'].str.contains('Baseline')]
    df = df[df.year>2005]

    ### get unique list of countries from FAO and GADM countries... need to rename some GADM countries to FAO designation 
    dfc = pd.read_csv( '/content/drive/My Drive/data/livestock/country_level_data/Production_Livestock_E_All_Data_NOFLAG.csv', sep = ',', encoding = 'iso-8859-1' )
    dfc = dfc[['Area','Item']].rename( columns = {'Area':'countryname'} )
    dfc = dfc[dfc.Item == 'Cattle']
    GADM =  pd.read_csv( '/content/drive/My Drive/data/area_masks/gadm36_country_names_and_abbr.csv', sep = ',' )
    GADM = GADM.rename( columns = {'NAME_0':'countryname', 'GID_0': 'ISO'} )
    GADM['maskid'] = GADM.index
    GADM = GADM[['ISO','maskid']]
    FAO =  pd.read_csv( '/content/drive/My Drive/data/area_masks/countrycodes_FAO_NAMES.csv', sep = ',' ).rename(columns = {'abb':'ISO'})
    FAO = FAO[['ISO','countryname']]
    countrynames = pd.merge( FAO, GADM, on = [ 'ISO' ] , how = 'inner' )
    smallisland = ['Barbados','Bermuda' ,'Dominica','Bahama','Grenada','Guam','Montserrat','Malta','Nauru','Niue','Saint Kitts and Nevis', 
                'Saint Pierre and Miquelon', 'St. Vincent and the Grenadines','Singapore','Tonga','Tuvalu','Wallis and Futuna Islands']
    dfc = dfc[ ~dfc.countryname.isin(smallisland) ]
    dfc = pd.merge( dfc, countrynames, on = [ 'countryname' ] , how = 'inner' )
    dfc = dfc.rename( columns = {'maskid': 'country'} ).drop( columns = ['Item'] )
    ######----------------------------------------------------------------------------------------------#####

    ### Land use data from LUH2 ###
    dfla = pd.read_csv( f'/content/drive/My Drive/data/livestock/land_use/ssp{ssp}_country_level.csv' ) ###ssp370
    dfla = pd.merge( dfla, dfc, on = 'country' ).drop(columns = ['countryname','country'])
    dfla = pd.merge(dfla, regions, on = 'ISO').rename( columns = {'R5_region':'REGION'})
    dfla['land'] = dfla.pastr + dfla.rang + dfla.crop

    dflabase = dfla[dfla.year == 2015]
    dflabase = dflabase[['ISO','land']].rename(columns ={ 'land': 'land2015'})
    dfla = pd.merge( dfla, dflabase, on = 'ISO' )
    dfla['landchange'] = dfla.land - dfla.land2015 

    dflaO= dfla[dfla.REGION == 'R5.2OECD']
    dflaregO = dflaO.groupby(['REGION','year'], as_index=False)['landchange'].sum().rename(columns ={ 'landchange': 'region_landchange'})
    dflaO = pd.merge( dflaO, dflaregO, on = ['REGION', 'year'] )
    dflaO['contribution'] = dflaO.landchange / dflaO.region_landchange

    dfla = dfla[dfla.REGION != 'R5.2OECD']
    dflareg = dfla[ dfla.landchange > 0 ] ### only take positive land change areas ###
    dflareg = dflareg.groupby(['REGION','year'], as_index=False)['landchange'].sum().rename(columns ={ 'landchange': 'region_landchange'})
    dfla = pd.merge( dfla, dflareg, on = ['REGION', 'year'] )
    dfla['contribution'] = dfla.landchange / dfla.region_landchange

    dfla = pd.concat( (dfla, dflaO) )

    decades = [2015, 2025, 2035, 2045, 2055, 2065, 2075, 2085, 2095]
    dfnew = pd.DataFrame({})
    for year in decades:
        dft = dfla[dfla.year>year]
        dft = dft[dft.year<year+10]
        dft = dft.groupby( ['ISO','Country','REGION'], as_index = False ).mean()
        dfnew = pd.concat( (dfnew, dft) )

    dfnew = dfnew.replace(2097.5, 2100)
    dfnew.year = dfnew.year.astype(int)
    dfnew.head(3)

    ### combine regional animal gains with country-level land change
    df = df[ df.SCENARIO == f'SSP{str(ssp)[0]}-Baseline' ] 
    df = pd.merge( dfnew, df, on = ['REGION','year'] )
    df['newcows'] = df.addcattle * df.contribution

    ### livestock numbers from FAO to get country level in 2010 again ###
    dfa = pd.read_csv( '/content/drive/My Drive/data/livestock/country_level_data/Environment_LivestockPatterns_E_All_Data_NOFLAG.csv', sep = ',', encoding = 'iso-8859-1' )
    dfa = dfa.drop( columns = ['Area Code', 'Element Code', 'Item Code'] )
    dfa = dfa.melt( id_vars = ['Area', 'Element', 'Item', 'Unit'], var_name = 'year', value_name = 'value' ).rename( columns = { 'Area' : 'Country' } )
    dfa.year = dfa.year.apply( lambda x: x.split('Y')[1] ).astype( int )
    dfa = dfa[dfa.Element == 'Stocks']
    dfa = dfa[dfa.Item.isin(['Cattle'])]
    dfa = dfa[dfa.Country != 'World']
    dfa = dfa[ dfa.year==2010 ]
    dfa = pd.merge( dfa, regions, on = ['Country'] )
    dfa = dfa[['ISO', 'value']].rename(columns={'value':'cows_country_2010'})
    dfa.cows_country_2010 = dfa.cows_country_2010 / 1e6

    df = pd.merge(df, dfa, on = ['ISO'])

    ### remove negative projections ###
    dfO = df[df.REGION == 'R5.2OECD']
    df = df[df.REGION != 'R5.2OECD']
    df.newcows[df.newcows < 0 ] = 0
    df = pd.concat(( df, dfO ))
    df['projected_cows'] = (df.newcows + df.cows_country_2010)

    ### load historical data ###
    root = '/content/drive/My Drive/data/livestock/cmip6_country/'
    files = os.listdir(root)
    files = fnmatch.filter(files, "*hist*HL*")
    files = [root + i for i in files]
    dft = pd.DataFrame()
    for file in files:
        df1 = pd.read_csv(file)
        df1['model'] = file.split('/')[-1].split('_')[0]
        dft = pd.concat((dft,df1))

    dfh = dft.groupby( ['country','model'], as_index = False ).mean().drop( columns = ['year'] ).rename(columns = {'produ':'pro_h'})
    dfhbase = dfh.groupby('country', as_index = False).mean()
    dfhbase.pro_h = dfhbase.pro_h.fillna(0)
    dfhbase = dfhbase.rename(columns = {'pro_h':'mean_hist_thi'})
    dfh = pd.merge( dfh, dfhbase, on =['country'] )

    files = os.listdir(root)
    files = fnmatch.filter(files, f"*{ssp}*HL*")
    files = [root + i for i in files]
    dft = pd.DataFrame()
    for file in files:
        df1 = pd.read_csv(file)
        df1['model'] = file.split('/')[-1].split('_')[0]
        dft = pd.concat( (dft, df1) )
    #df['ssp'] = 'SSP5-85'

    dft = pd.merge( dft, dfh,  on = ['country','model'], how = 'left' )
    dft = pd.merge( dft, dfc, on =['country'], how = 'left').drop(columns =['country'] )
    dft.year = dft.year + 5

    ### annoying issue with 2095-2100 not being the same as the other decades ###
    dft = dft[dft.model != 'MRI-ESM2-0']
    df2090 = dft[dft.year != 2100]
    df2100 = dft[dft.year == 2100]
    df2100.produ *= 1.67
    dft = pd.concat( ( df2090, df2100 ) )

    dft['heat_increase'] = dft.produ - dft.pro_h
    dft = dft.dropna()
    dft = dft[['year', 'ISO','heat_increase','model', 'mean_hist_thi']]

    ### merge with cow data ###
    df = pd.merge(df, dft, on = ['year','ISO'])
    return df


In [ ]:
df = get_full_ssp(370)
dft = df[['year', 'ISO','Country','REGION','cattle','newcows','heat_increase','mean_hist_thi','model','cows_country_2010', 'projected_cows']]
dft.heat_increase = dft.heat_increase / 3650 ## decadal to daily
dft.mean_hist_thi = dft.mean_hist_thi / 3650
dft['risk'] = (dft.projected_cows * dft.heat_increase) / 1000
dft['risk2010'] = (dft.cows_country_2010 * dft.heat_increase) / 1000
dft['risk2010_hist'] = (dft.cows_country_2010 * dft.mean_hist_thi) / 1000
df370 = dft.copy()
df370 = df370.groupby(['model','REGION','year'], as_index= False ).sum()
df370['per_change'] = df370.risk / df370.risk2010_hist

In [ ]:
df = get_full_ssp(126)
dft = df[['year', 'ISO','Country','REGION','cattle','newcows','heat_increase','mean_hist_thi','model','cows_country_2010', 'projected_cows']]
dft.heat_increase = dft.heat_increase / 3650 ## decadal to daily
dft.mean_hist_thi = dft.mean_hist_thi / 3650
dft['risk'] = (dft.projected_cows * dft.heat_increase) / 1000
dft['risk2010'] = (dft.cows_country_2010 * dft.heat_increase) / 1000
dft['risk2010_hist'] = (dft.cows_country_2010 * dft.mean_hist_thi) / 1000
df126 = dft.copy()
df126 = df126.groupby(['model','REGION','year'], as_index= False ).sum()
df126['per_change'] = (df126.risk) / df126.risk2010_hist

In [ ]:
colors = [rgb2hex(i) for i in px.colors.sequential.deep[2::3]]

plt.close('all')
fig = plt.figure( figsize = (10, 4.5 ))
gs = GridSpec( 2, 4, height_ratios = [2,0.75] ) 
gs.update(wspace = 0.05, hspace = 0.05)

newyears = np.arange( 2020, 2101, 1)
k = 2

def make_subplot( region, colorid, ylim, ticks, xticks, yaxis=True):
    dft = df370[df370.REGION == region]
    models = dft.model.unique()
    for model in models:
        dft1 = dft[dft.model == model]
        spl1 = make_interp_spline(dft1.year.values, dft1.per_change.values, k=k)
        plt.plot( newyears, spl1(newyears),  lw = 0.4, color = colors[colorid], zorder = 3)

    years = [2020,2030,2040,2050,2060,2070,2080,2090,2100]
    riskmax = []
    riskmin = []
    riskmed = []
    for year in years:
        dft1 = dft[dft.year == year]
        riskmax.append( dft1.per_change.values.max() )
        riskmin.append( dft1.per_change.values.min() )
        riskmed.append( dft1.per_change.values.mean())

    spl1 = make_interp_spline( years, riskmin, k=k )
    spl2 = make_interp_spline( years, riskmax, k=k )
    plt.fill_between(newyears, y1 = spl1(newyears),   y2 = spl2(newyears), facecolor = colors[colorid], edgecolor = 'none', alpha = 0.2, zorder = 2 )
    spl1 = make_interp_spline( years, riskmed, k=k )
    plt.plot(newyears, spl1(newyears),   lw = 1.4, color = 'black', zorder = 5)

    spl1 = make_interp_spline( years, riskmin, k=k )
    spl2 = make_interp_spline( years, riskmax, k=k )
    plt.fill_between(newyears, y1 = spl1(newyears),   y2 = spl2(newyears), facecolor = '#bdbdbd', edgecolor = 'none', alpha = 0.4, zorder = 0 )
    spl1 = make_interp_spline( years, riskmed, k=k )
    plt.plot(newyears, spl1(newyears),   lw = 1.4, color = 'black', zorder = 2)

    plt.xticks([2020,2040,2060,2080,2100], xticks)
    plt.ylim(0,ylim)
    plt.yticks(ticks, ticks)

    if yaxis == False: 
        ax1.axes.get_yaxis().set_visible(False)
        plt.yticks([1], [])
        sns.despine( offset = 3, trim = True )
    else:
        sns.despine( offset = 3, trim = True )

ax1 = plt.subplot(gs[0,0])
make_subplot( 'R5.2OECD', 0, 10, [0,1,2,3,4,5,6,7,8,9,10],[0,1,2,3,4,5,6,7,8,9,10])

ax1 = plt.subplot(gs[0,1])
make_subplot( 'R5.2ASIA', 1, 10, [],[], False )

ax1 = plt.subplot(gs[0,2])
make_subplot( 'R5.2LAM', 2, 10, [],[], False )

ax1 = plt.subplot(gs[0,3])
make_subplot( 'R5.2MAF', 3, 10, [],[], False )

ax1 = plt.subplot(gs[1,0])
dft = df370[df370.REGION == 'R5.2OECD']
years = [2020,2030,2040,2050,2060,2070,2080,2090,2100]
riskmed = []
riskmed2010 = []
for year in years:
    dft1 = dft[dft.year == year]
    riskmed.append( dft1.risk.values.mean() )
    riskmed2010.append( dft1.risk2010.values.mean() )

spl1 = make_interp_spline( years, riskmed, k=k )
spl2 = make_interp_spline( years, riskmed2010, k=k )
plt.fill_between(newyears, y1 = 0,  y2 = spl2(newyears), facecolor = colors[0], edgecolor = 'none', alpha = 0.2, zorder = 2 )
plt.fill_between(newyears, y1 = spl2(newyears),  y2 = spl1(newyears), facecolor = colors[0], edgecolor = 'none', alpha = 0.5, zorder = 2 )
plt.plot(newyears, spl1(newyears), lw = 1.4, color = 'black', zorder = 2)

plt.ylim(0,4.5)
plt.xticks([2020,2040,2060,2080,2100], [2020,2040,2060,2080,2100])
ax1.axes.get_yaxis().set_visible(False)
plt.yticks([1], [])
sns.despine( offset = 3, trim = True )

ax1 = plt.subplot(gs[1,1])
dft = df370[df370.REGION == 'R5.2ASIA']
years = [2020,2030,2040,2050,2060,2070,2080,2090,2100]
riskmed = []
riskmed2010 = []
for year in years:
    dft1 = dft[dft.year == year]
    riskmed.append( dft1.risk.values.mean() )
    riskmed2010.append( dft1.risk2010.values.mean() )

spl1 = make_interp_spline( years, riskmed, k=k )
spl2 = make_interp_spline( years, riskmed2010, k=k )
plt.fill_between(newyears, y1 = 0,  y2 = spl2(newyears), facecolor = colors[1], edgecolor = 'none', alpha = 0.2, zorder = 2 )
plt.fill_between(newyears, y1 = spl2(newyears),  y2 = spl1(newyears), facecolor = colors[1], edgecolor = 'none', alpha = 0.5, zorder = 2 )
plt.plot(newyears, spl1(newyears), lw = 1.4, color = 'black', zorder = 2)

plt.ylim(0,4.5)
plt.xticks([2020,2040,2060,2080,2100], [2020,2040,2060,2080,2100])
ax1.axes.get_yaxis().set_visible(False)
plt.yticks([1], [])
sns.despine( offset = 3, trim = True )

ax1 = plt.subplot(gs[1,2])
dft = df370[df370.REGION == 'R5.2LAM']
years = [2020,2030,2040,2050,2060,2070,2080,2090,2100]
riskmed = []
riskmed2010 = []
for year in years:
    dft1 = dft[dft.year == year]
    riskmed.append( dft1.risk.values.mean() )
    riskmed2010.append( dft1.risk2010.values.mean() )

spl1 = make_interp_spline( years, riskmed, k=k )
spl2 = make_interp_spline( years, riskmed2010, k=k )
plt.fill_between(newyears, y1 = 0,  y2 = spl2(newyears), facecolor = colors[2], edgecolor = 'none', alpha = 0.2, zorder = 2 )
plt.fill_between(newyears, y1 = spl2(newyears),  y2 = spl1(newyears), facecolor = colors[2], edgecolor = 'none', alpha = 0.5, zorder = 2 )
plt.plot(newyears, spl1(newyears), lw = 1.4, color = 'black', zorder = 2)

plt.ylim(0,4.5)
plt.xticks([2020,2040,2060,2080,2100], [2020,2040,2060,2080,2100])
ax1.axes.get_yaxis().set_visible(False)
plt.yticks([1], [])
sns.despine( offset = 3, trim = True )

ax1 = plt.subplot(gs[1,3])
dft = df370[df370.REGION == 'R5.2MAF']
years = [2020,2030,2040,2050,2060,2070,2080,2090,2100]
riskmed = []
riskmed2010 = []
for year in years:
    dft1 = dft[dft.year == year]
    riskmed.append( dft1.risk.values.mean() )
    riskmed2010.append( dft1.risk2010.values.mean() )

spl1 = make_interp_spline( years, riskmed, k=k )
spl2 = make_interp_spline( years, riskmed2010, k=k )
plt.fill_between(newyears, y1 = 0,  y2 = spl2(newyears), facecolor = colors[3], edgecolor = 'none', alpha = 0.2, zorder = 2 )
plt.fill_between(newyears, y1 = spl2(newyears),  y2 = spl1(newyears), facecolor = colors[3], edgecolor = 'none', alpha = 0.5, zorder = 2 )
plt.plot(newyears, spl1(newyears), lw = 1.4, color = 'black', zorder = 2)

plt.ylim(0,4.5)
plt.xticks([2020,2040,2060,2080,2100], [2020,2040,2060,2080,2100])
ax1.axes.get_yaxis().set_visible(False)
plt.yticks([1], [])
sns.despine( offset = 3, trim = True )

plt.savefig( '/content/drive/My Drive/figures/livestock/figure_4_main.png', dpi = 900, bbox_inches = 'tight', pad_inches = 0.00 )

## Supplemental figure, livestock share

In [ ]:
df = pd.read_csv( '/content/drive/My Drive/data/livestock/country_level_data/Environment_LivestockPatterns_E_All_Data_NOFLAG.csv', sep = ',', encoding = 'iso-8859-1' )
df = df.drop( columns = ['Area Code', 'Element Code', 'Item Code'] )
df = df.melt( id_vars = ['Area', 'Element', 'Item', 'Unit'], var_name = 'year', value_name = 'value' ).rename( columns = { 'Area' : 'Country' } )
df.year = df.year.apply( lambda x: x.split('Y')[1] ).astype( int )
df = df[df.Unit == '% of total LSU']
region = 'World'
df = df[ df.Country == region ]
ruminants = ['Asses', 'Buffaloes', 'Camels', 'Cattle', 'Chickens', 'Goats','Horses', 'Mules', 'Pigs', 'Sheep']
df = df[df.Item.isin(ruminants)]
animals = df.Item.unique()

plt.close('all')
fig = plt.figure( figsize = (4.5, 4 ))

colors = px.colors.qualitative.Plotly
for i, animal in enumerate(animals):
    dft = df[df.Item == animal]
    #np.lins
    plt.plot(dft.year, dft.value, color = colors[i])
    plt.annotate(animal, xy = (dft.year.values[-1]+1, dft.value.values[-1]-0.01), fontsize = 6, color = colors[i])

plt.xlim(1960, 2030)
plt.xticks([1960,1980,2000,2020], [1960,1980,2000,2020])
plt.ylabel(f'Share of total livestock in {region} (%)')
plt.xlabel('Year')
#plt.ylim(0,1)
sns.despine( offset = 3, trim = True )
plt.yticks(rotation = 90, va = 'center')

plt.savefig( f'/content/drive/My Drive/figures/livestock/cattle_share_{region}.png', dpi = 900, bbox_inches = 'tight', pad_inches = 0.00 )

In [ ]:
df = pd.read_csv( '/content/drive/My Drive/data/livestock/country_level_data/Environment_LivestockPatterns_E_All_Data_NOFLAG.csv', sep = ',', encoding = 'iso-8859-1' )
df = df.drop( columns = ['Area Code', 'Element Code', 'Item Code'] )
df = df.melt( id_vars = ['Area', 'Element', 'Item', 'Unit'], var_name = 'year', value_name = 'value' ).rename( columns = { 'Area' : 'Country' } )
df.year = df.year.apply( lambda x: x.split('Y')[1] ).astype( int )
df = df[df.Unit == 'LSU/ha']
ruminants = ['Cattle']
df = df[df.Item.isin(ruminants)]
df = df.groupby(['Country','year'], as_index= False).sum()
df = df[df.value > 0]

colors = [rgb2hex(i) for i in px.colors.sequential.deep_r]

plt.close('all')
fig = plt.figure( figsize = (4.5, 4 ))

regions = ['Africa','South America','United States of America', 'United Kingdom','Ethiopia', 'Australia',  'Zimbabwe', 'China', 'India', 'Brazil', 'Italy', 'Japan']
for i, region in enumerate( regions ):
    dft = df[df.Country == region]
    plt.plot(dft.year, dft.value, color = colors[i])
    plt.annotate(region[:12], xy = (dft.year.values[-1]+1, dft.value.values[-1]-0.01), fontsize = 6, color = colors[i])

plt.xlim(1960, 2030)
plt.xticks([1960,1980,2000,2020], [1960,1980,2000,2020])
plt.ylabel('Ruminants per ha (of ag. lang)')
plt.xlabel('Year')
plt.ylim(0,1)
sns.despine( offset = 3, trim = True )
plt.yticks(rotation = 90, va = 'center')

plt.savefig( '/content/drive/My Drive/figures/livestock/cattle_densities_timeseries.png', dpi = 900, bbox_inches = 'tight', pad_inches = 0.00 )

# Process land use data

In [ ]:
### Process the LUH2 data to country level ###
ssp585 = '/content/drive/My Drive/data/livestock/land_use/multiple-states_input4MIPs_landState_ScenarioMIP_UofMD-MAGPIE-ssp585-2-1-f_gn_2015-2100.nc'
ssp370 = '/content/drive/My Drive/data/livestock/land_use/multiple-states_input4MIPs_landState_ScenarioMIP_UofMD-AIM-ssp370-2-1-f_gn_2015-2100.nc'
ssp126 = '/content/drive/My Drive/data/livestock/land_use/multiple-states_input4MIPs_landState_ScenarioMIP_UofMD-IMAGE-ssp126-2-1-f_gn_2015-2100.nc'

mask = xr.load_dataarray('/content/drive/Shared drives/ClimateAg/data/gadm36_level0_0_25_180_180.nc4')
mask = np.flip( np.nan_to_num( mask.values ), axis = 0 )

x = xr.open_dataset(ssp126, decode_times=False)
pastr = np.nan_to_num( x.pastr.values ) * cellarea
rang  = np.nan_to_num( x.range.values ) * cellarea
c3  = np.nan_to_num( x.c3ann.values )   * cellarea
c4  = np.nan_to_num( x.c4ann.values )   * cellarea
c3n  = np.nan_to_num( x.c3nfx.values )  * cellarea
crop = c3 + c4 + c3n

years = np.arange( 2015, 2100, 1 )
df = []
for i in range( 85 ):
    print(i)
    for j in np.unique( mask ):
        df.append({'year':  years[i], 'country': j, 
                   'pastr': np.ma.masked_where(mask != j, pastr[i,:,:]).sum(),
                   'rang':  np.ma.masked_where(mask != j, rang[i,:,:]).sum(),
                   'crop':  np.ma.masked_where(mask != j, crop[i,:,:]).sum()})

df = pd.DataFrame(df)
df.to_csv('/content/drive/My Drive/data/livestock/land_use/ssp126_country_level.csv', index = False )



---

# OLD

---



In [ ]:
### SSP projections from IIASA ###
iam = pd.read_csv('/content/drive/My Drive/data/econ/ssps/SSP_IAM_V2_201811.csv')
iam = iam[iam.VARIABLE == 'Agricultural Demand|Livestock']
iam = iam.melt( id_vars = ['MODEL', 'SCENARIO', 'REGION', 'VARIABLE', 'UNIT'], var_name = 'year', value_name = 'value' )
iam = iam.groupby( by = ['SCENARIO', 'year', 'REGION'], as_index = False ).mean()
iam.year = pd.to_numeric( iam.year )

### get percent change from 2010 baseline ###
iam2010 = iam[ iam.year == 2010 ]
iam2010 = iam2010.drop( columns = ('year') ).rename( columns = {'value':'baseline'} )
iam = pd.merge( iam, iam2010, on = ['SCENARIO', 'REGION'] )
iam['change'] = (iam.value - iam.baseline) / iam.baseline * 100
iam['ssp']  = iam.SCENARIO.apply( lambda x: x.split('-')[0] )
iam = iam.drop( columns = ['value', 'baseline'] )

### region designations from FAO ###
regions = pd.read_excel('/content/drive/My Drive/data/econ/ssps/cmip6_iam_model_region_mapping.xlsx')
regions = regions[ ['ISO', 'Country', 'R5_region'] ]
regions= regions.replace( ['R5LAM', 'R5MAF', 'R5REF', 'R5OECD', 'R5ASIA','United States'], ['R5.2LAM', 'R5.2MAF', 'R5.2REF', 'R5.2OECD', 'R5.2ASIA','United States of America'] )

### livestock numbers from FAO ###
df = pd.read_csv( '/content/drive/My Drive/data/livestock/country_level_data/Environment_LivestockPatterns_E_All_Data_NOFLAG.csv', sep = ',', encoding = 'iso-8859-1' )
df = df.drop( columns = ['Area Code', 'Element Code', 'Item Code'] )
df = df.melt( id_vars = ['Area', 'Element', 'Item', 'Unit'], var_name = 'year', value_name = 'value' ).rename( columns = { 'Area' : 'Country' } )
df.year = df.year.apply( lambda x: x.split('Y')[1] ).astype( int )
df = df[df.Element == 'Stocks']
ruminants = ['Goat','Sheep','Cattle','Buffaloes']

df = df[df.Item.isin(ruminants)]
df = df[df.Country != 'World']
df = pd.merge( df, regions, on = ['Country'] ).rename( columns = {'R5_region':'REGION'})
df = df[df.year==2015]
df = df.drop( columns = ('year') )
df = df.groupby(['Country','REGION','ISO'], as_index=False).sum().rename(columns = {'value':'runminants'})

### Land use data from LUH2 ###
dfla = pd.read_csv( '/content/drive/My Drive/data/livestock/land_use/ssp370_country_level.csv' )
dfla = pd.merge( dfla, dfc, on = 'country' ).drop(columns = ['countryname','country'])
dfla = dfla[dfla.year == 2015]
df = pd.merge( df, dfla, on = 'ISO')

df['land'] = df.pastr + df.rang + df.crop
df['density'] = df.runminants / df.land
df = df[df.density< 100]
df = d
df.sort_values( by = 'density', ascending = False )

In [ ]:
plt.hist(df.density, bins = 10)

In [ ]:
dfl =  pd.read_csv( '/content/drive/My Drive/data/livestock/land_use/ssp370_country_level.csv' )
dfl = dfl[dfl.year == 2015]
dfh = dfh.drop( columns = ['year','pastr','rang']).rename(columns = {'land':'land_h'})
df = pd.merge( df, dfh, on = 'country')
df['per_change'] = (df.land - df.land_h) / df.land_h
df = pd.merge( df, dfc, on = 'country' )
dft = df[df.year == 2015]
dft.land.sum()

In [ ]:
### SSP projections from IIASA ###
iam = pd.read_csv('/content/drive/My Drive/data/econ/ssps/SSP_IAM_V2_201811.csv')
iam = iam[iam.VARIABLE == 'Agricultural Demand|Livestock']
iam = iam.melt( id_vars = ['MODEL', 'SCENARIO', 'REGION', 'VARIABLE', 'UNIT'], var_name = 'year', value_name = 'value' )
iam = iam.groupby( by = ['SCENARIO', 'year', 'REGION'], as_index = False ).mean()
iam.year = pd.to_numeric( iam.year )

### get percent change from 2010 baseline ###
iam2010 = iam[ iam.year == 2010 ]
iam2010 = iam2010.drop( columns = ('year') ).rename( columns = {'value':'baseline'} )
iam = pd.merge( iam, iam2010, on = ['SCENARIO', 'REGION'] )
iam['change'] = (iam.value - iam.baseline) / iam.baseline * 100
iam['ssp']  = iam.SCENARIO.apply( lambda x: x.split('-')[0] )
iam = iam.drop( columns = ['value', 'baseline'] )

### region designations from FAO ###
regions = pd.read_excel('/content/drive/My Drive/data/econ/ssps/cmip6_iam_model_region_mapping.xlsx')
regions = regions[ ['ISO', 'Country', 'R5_region'] ]
regions= regions.replace( ['R5LAM', 'R5MAF', 'R5REF', 'R5OECD', 'R5ASIA'], ['R5.2LAM', 'R5.2MAF', 'R5.2REF', 'R5.2OECD', 'R5.2ASIA'] )

### livestock numbers from FAO ###
df = pd.read_csv( '/content/drive/My Drive/data/livestock/country_level_data/Environment_LivestockPatterns_E_All_Data_NOFLAG.csv', sep = ',', encoding = 'iso-8859-1' )
df = df.drop( columns = ['Area Code', 'Element Code', 'Item Code'] )
df = df.melt( id_vars = ['Area', 'Element', 'Item', 'Unit'], var_name = 'year', value_name = 'value' ).rename( columns = { 'Area' : 'Country' } )
df.year = df.year.apply( lambda x: x.split('Y')[1] ).astype( int )
df = df[df.Element == 'Stocks']
df = df[df.Item == 'Cattle']
df = df[df.Country != 'World']
df = pd.merge( df, regions, on = ['Country'] ).rename( columns = {'R5_region':'REGION'})
df = df[df.year==2010]
df = df.drop( columns = ('year') )

### Combine country level livestock data with regional ssp projected changes ###
df = pd.merge(df, iam, on = ['REGION'])
df['livestock'] = df.value * (1 + df.change / 100)
dfL = df[ ['Country','ISO','REGION','SCENARIO','year','livestock'] ]
dfL = dfL.rename( columns = { 'SCENARIO': 'ssp' } )
dfb = dfL[ dfL.year == 2020 ]
dfb = dfb.rename(columns={'livestock':'livestock2020'}).drop(columns = 'year')
dfL = pd.merge( dfL, dfb, on = ['Country','ISO','REGION','ssp'] )
dfL = dfL.replace( ['SSP5-Baseline', 'SSP3-Baseline'], ['SSP5-85', 'SSP3-70'] )
dfL = dfL[ dfL.year > 2010 ]

In [ ]:
### region designations from FAO ###
regions = pd.read_excel('/content/drive/My Drive/data/econ/ssps/cmip6_iam_model_region_mapping.xlsx')
regions = regions[ ['ISO', 'Country', 'R5_region'] ]
regions= regions.replace( ['R5LAM', 'R5MAF', 'R5REF', 'R5OECD', 'R5ASIA'], ['R5.2LAM', 'R5.2MAF', 'R5.2REF', 'R5.2OECD', 'R5.2ASIA'] )

df = pd.read_csv( '/content/drive/My Drive/data/livestock/country_level_data/Environment_LivestockPatterns_E_All_Data_NOFLAG.csv', sep = ',', encoding = 'iso-8859-1' )
df = df.drop( columns = ['Area Code', 'Element Code', 'Item Code'] )
df = df.melt( id_vars = ['Area', 'Element', 'Item', 'Unit'], var_name = 'year', value_name = 'value' ).rename( columns = { 'Area' : 'Country' } )
df.year = df.year.apply( lambda x: x.split('Y')[1] ).astype( int )
df = df[df.Element == 'Stocks']
df = df[df.Item == 'Cattle']
df = df[df.Country != 'World']
df = pd.merge( df, regions, on = ['Country'] ).rename( columns = {'R5_region':'REGION'})
df = df[df.year==2017]
df = df.drop( columns = ('year') )
df = df[df.Country == 'Central African Republic']
df

In [ ]:
502692.5 * 8

## country names

In [ ]:
### get unique list of countries from FAO and GADM countries... need to rename some GADM countries to FAO designation 
df = pd.read_csv( '/content/drive/My Drive/data/livestock/country_level_data/Production_Livestock_E_All_Data_NOFLAG.csv', sep = ',', encoding = 'iso-8859-1' )
df = df[['Area','Item']].rename( columns = {'Area':'countryname'} )
df = df[df.Item == 'Cattle']
GADM =  pd.read_csv( '/content/drive/My Drive/data/area_masks/gadm36_country_names_and_abbr.csv', sep = ',' )
GADM = GADM.rename( columns = {'NAME_0':'countryname', 'GID_0': 'ISO'} )
GADM['maskid'] = GADM.index
GADM = GADM[['ISO','maskid']]
FAO =  pd.read_csv( '/content/drive/My Drive/data/area_masks/countrycodes_FAO_NAMES.csv', sep = ',' ).rename(columns = {'abb':'ISO'})
FAO = FAO[['ISO','countryname']]
countrynames = pd.merge( FAO, GADM, on = [ 'ISO' ] , how = 'inner' )
smallisland = ['Barbados','Bermuda' ,'Dominica','Bahama','Grenada','Guam','Montserrat','Malta','Nauru','Niue','Saint Kitts and Nevis', 
               'Saint Pierre and Miquelon', 'St. Vincent and the Grenadines','Singapore','Tonga','Tuvalu','Wallis and Futuna Islands']
df = df[ ~df.countryname.isin(smallisland) ]
dfc = pd.merge( df, countrynames, on = [ 'countryname' ] , how = 'inner' )
countrynames = dfc.maskid.unique()

dfc = dfc.rename( columns = {'maskid': 'country'} ).drop( columns = ['Item'] )

## temperature data

In [ ]:
root = '/content/drive/My Drive/data/livestock/cmip6_country/'
files = os.listdir(root)
files = fnmatch.filter(files, "*hist*")
files = [root + i for i in files]
df = pd.DataFrame()
for file in files:
    df1 = pd.read_csv(file)
    df1['model'] = file.split('/')[-1].split('_')[0]
    df = pd.concat((df,df1))

dfh = df.groupby( ['country','model'], as_index = False ).mean().drop( columns = ['year'] ).rename(columns = {'produ':'pro_h','death':'dea_h'})

files = os.listdir(root)
files = fnmatch.filter(files, "*585*")
files = [root + i for i in files]
df = pd.DataFrame()
for file in files:
    df1 = pd.read_csv(file)
    df1['model'] = file.split('/')[-1].split('_')[0]
    df = pd.concat((df,df1))
#df['ssp'] = 'SSP5-85'

df = pd.merge( df, dfh,  on = ['country','model'], how = 'left' )
df = pd.merge( df, dfc, on =['country'], how = 'left').drop(columns =['country'] )
df.year = df.year + 5

### annoying issue with 2095-2100 not being the same as the other decades ###
df2090 = df[df.year != 2100]
df2100 = df[df.year == 2100]
df2100b = df2100[df2100.model == 'MRI-ESM2-0']
df2100 = df2100[df2100.model != 'MRI-ESM2-0']
df2100.produ *= 1.67 # six years... except for MRI, which is the full decade

df = pd.concat( ( df2090, df2100, df2100b ) )
df['heat_incease'] = df.produ - df.pro_h
df = df.dropna()

## merge temperature and animal data

In [ ]:
dfL = dfL[dfL.ssp == 'SSP5-85']
df = pd.merge(df, dfL, on = ['ISO', 'year'])

df['risk'] = df.heat_incease * df.livestock
df['risk2020'] = df.heat_incease * df.livestock2020
df = df[ ['REGION','ISO','model','year','livestock','livestock2020','risk','risk2020', 'heat_incease'] ]

dfr = df.groupby( ['REGION','model','year'], as_index = False ).sum()
dfr.head(5)

In [ ]:
df = df.groupby( ['REGION','ISO','year'], as_index = False ).mean()

In [ ]:
df = df[df.year == 2090]
df['live_increase'] = df.livestock-df.livestock2020

In [ ]:
regions = ['R5.2OECD','R5.2ASIA', 'R5.2LAM', 'R5.2MAF']

plt.rcParams.update( {'font.size': 8, 'xtick.labelsize' : 6, 'ytick.labelsize' : 6, 'legend.fontsize': 6, 'legend.frameon': False, 
                     'axes.linewidth':0.4, 'xtick.major.width':0.4, 'ytick.major.width':0.4, 'xtick.major.size':1, 'ytick.major.size':1,
                     'xtick.bottom': True, 'xtick.top': False, 'ytick.right': False, 'ytick.right': False, 'legend.numpoints': 4,
                     'xtick.major.pad':2.5, 'ytick.major.pad':2.5, 'axes.labelpad':2,'xtick.direction': 'out', 'ytick.direction': 'out', 
                      'axes.edgecolor':'black','ytick.color':'black','xtick.color':'black','axes.facecolor':'white'} )


plt.close('all')
fig = plt.figure( figsize = (5, 5 ), frameon=True)

for i, region in enumerate(regions):
    dft1 = df[df.REGION == region]
    for index, row in dft1.iterrows():
        if row.livestock > 30000000: 
            plt.annotate(row.ISO, xy = (row.heat_incease/3650-0.2, row.livestock/1e6-row.livestock2020/1e6 - 1), fontsize = 5, zorder = 10 )
            plt.scatter(row.heat_incease/3650, row.livestock/1e6-row.livestock2020/1e6, facecolors = colors[i], edgecolors='black', lw =0.8, alpha = 0.4, s = np.sqrt(row.livestock/1e6) * 40)
        else: 
            plt.scatter(row.heat_incease/3650, row.livestock/1e6-row.livestock2020/1e6, facecolors = colors[i], edgecolors='gray', lw =0.2, alpha = 0.4, s = np.sqrt(row.livestock/1e6) * 40) 

plt.scatter(0.5,90, s = np.sqrt(100) * 40, facecolors = 'gray', edgecolors='gray', lw =0.2, alpha = 0.3 )
plt.annotate('100M', xy=(0.9, 89), fontsize = 7)
plt.scatter(0.5,80, s = np.sqrt(10) * 40, facecolors = 'gray', edgecolors='gray', lw =0.2, alpha = 0.3 )
plt.annotate('10M', xy=(0.9, 79), fontsize = 7)
plt.scatter(0.5,70, s = np.sqrt(1) * 40, facecolors = 'gray', edgecolors='gray', lw =0.2, alpha = 0.3 )
plt.annotate('1M ', xy=(0.9, 69), fontsize = 7)

plt.xticks([0,1,2,3,4,5,6,7,8],[0,1,2,3,4,5,6,7,8] )
plt.yticks([0,20,40,60,80,100],[0,20,40,60,80,100] )
sns.despine( offset = 3, trim = True )
plt.yticks(rotation = 90, va = 'center')
plt.xlabel('Heat increase (mean THI over threshold)')
plt.ylabel('Livestock increase (Million head)')

plt.savefig( '/content/drive/My Drive/figures/livestock/figure_4.png', dpi = 500, bbox_inches = 'tight', pad_inches = 0.00 )

In [ ]:
150000000/1104370

In [ ]:
223984029/8516000

# Figure 4

In [ ]:
fig = plt.figure( figsize = (3.5, 5 ))

In [ ]:
[rgb2hex(i) for i in px.colors.sequential.deep[1::3]]

In [ ]:
colors = ['#3c78d8', '#6aa84f', '#e69138', '#a61c00','#674ea7']
colors = [rgb2hex(i) for i in px.colors.sequential.deep[1::3]]


plt.rcParams.update( {'font.size': 8, 'xtick.labelsize' : 6, 'ytick.labelsize' : 6, 'legend.fontsize': 6, 'legend.frameon': False, 
                     'axes.linewidth':0.4, 'xtick.major.width':0.4, 'ytick.major.width':0.4, 'xtick.major.size':1, 'ytick.major.size':1,
                     'xtick.bottom': True, 'xtick.top': False, 'ytick.right': False, 'ytick.right': False, 'legend.numpoints': 4,
                     'xtick.major.pad':2.5, 'ytick.major.pad':2.5, 'axes.labelpad':2,'xtick.direction': 'out', 'ytick.direction': 'out', 
                      'axes.edgecolor':'black','ytick.color':'black','xtick.color':'black','axes.facecolor':'white'} )


plt.close('all')
fig = plt.figure( figsize = (9, 2.25 ))
gs = GridSpec(1, 4) # height_ratios = [1, 0.05], width_ratios = [0.4, 1, 0.05] )
gs.update(wspace = 0.1, hspace = 0.0)

newyears = np.arange(2020,2101,1)
k = 2

def make_subplot( region, colorid, yaxis=True ):
    factor = 3650e9
    dft = dfr[dfr.REGION == region]
    models = dft.model.unique()
    for model in models:
        dft1 = dft[dft.model == model]
        
        spl1 = make_interp_spline(dft1.year, dft1.risk/factor, k=k)
        spl2 = make_interp_spline(dft1.year, dft1.risk2020/factor, k=k)
        plt.plot(newyears, spl1(newyears),     lw = 0.4, color = colors[colorid], zorder = 3)
        plt.plot(newyears, spl2(newyears), lw = 0.4, color = '#d9d9d9', zorder = 1)

    years = [2020,2030,2040,2050,2060,2070,2080,2090,2100]
    riskmax = []
    riskmin = []
    riskmax20 = []
    riskmin20 = []
    riskmed = []
    riskmed20 = []
    for year in years:
        dft1 = dft[dft.year == year]
        riskmax.append( dft1.risk.values.max()/factor )
        riskmin.append( dft1.risk.values.min()/factor )
        riskmax20.append( dft1.risk2020.values.max()/factor )
        riskmin20.append( dft1.risk2020.values.min()/factor )
        riskmed.append( dft1.risk.values.mean()/factor )
        riskmed20.append( dft1.risk2020.values.mean()/factor )

    spl1 = make_interp_spline( years, riskmin, k=k )
    spl2 = make_interp_spline( years, riskmax, k=k )
    spl3 = make_interp_spline( years, riskmin20, k=k )
    spl4 = make_interp_spline( years, riskmax20, k=k )

    plt.fill_between(newyears, y1 = spl1(newyears),   y2 = spl2(newyears), facecolor = colors[colorid], edgecolor = 'none', alpha = 0.2, zorder = 2 )
    plt.fill_between(newyears, y1 = spl3(newyears), y2 = spl4(newyears), facecolor = '#d9d9d9', edgecolor = 'none', alpha = 0.3, zorder = 0 )

    spl1 = make_interp_spline( years, riskmed, k=k )
    spl2 = make_interp_spline( years, riskmed20, k=k)

    plt.plot(newyears, spl1(newyears),   lw = 1.4, color = 'black', zorder = 5)
    plt.plot(newyears, spl2(newyears), lw = 1.4, color = '#525252', zorder = 4)

    plt.xticks([2020,2040,2060,2080,2100], [2020,2040,2060,2080,2100])

    plt.ylim(0,6.7)
    plt.yticks([0,1,2,3,4,5,6], [0,1,2,3,4,5,6])

    plt.annotate(region, xy = (2020,2), color = colors[colorid], fontsize = 7)
    if yaxis == False: 
        ax1.axes.get_yaxis().set_visible(False)
        plt.yticks([0], [])
    else:
        plt.yticks(rotation = 90, va = 'center')

    sns.despine( offset = 3, trim = True )

ax1 = plt.subplot(gs[0,0])
make_subplot( 'R5.2OECD', 0)

ax1 = plt.subplot(gs[0,1])
make_subplot( 'R5.2ASIA', 1, False)

ax1 = plt.subplot(gs[0,2])
make_subplot( 'R5.2LAM', 2, False)

ax1 = plt.subplot(gs[0,3])
make_subplot( 'R5.2MAF', 3, False)

plt.savefig( '/content/drive/My Drive/figures/livestock/figure_4.png', dpi = 900, bbox_inches = 'tight', pad_inches = 0.00 )

In [ ]:
root = '/content/drive/My Drive/data/livestock/cmip6_country/'
files = os.listdir(root)
files = fnmatch.filter(files, "*hist*")
files = [root + i for i in files]
df = pd.DataFrame()
for file in files:
    df1 = pd.read_csv(file)
    df1['model'] = file.split('/')[-1].split('_')[0]
    df = pd.concat((df,df1))

dfh = df.groupby( ['country','model'], as_index = False ).mean().drop( columns = ['year'] ).rename(columns = {'produ':'pro_h','death':'dea_h'})

files = os.listdir(root)
files = fnmatch.filter(files, "*245*")
files = [root + i for i in files]
df = pd.DataFrame()
for file in files:
    df1 = pd.read_csv(file)
    df1['model'] = file.split('/')[-1].split('_')[0]
    df = pd.concat((df,df1))
df['ssp'] = 'SSP2-45'

df = pd.merge( df, dfh, on = ['country','model'], how = 'left' )
df2 = pd.merge( df, dfc, on =['country'], how = 'left').drop(columns =['country'] )
df2.year = df2.year + 5

files = os.listdir(root)
files = fnmatch.filter(files, "*370*")
files = [root + i for i in files]
df = pd.DataFrame()
for file in files:
    df1 = pd.read_csv(file)
    df1['model'] = file.split('/')[-1].split('_')[0]
    df = pd.concat((df,df1))
df['ssp'] = 'SSP3-70'

df = pd.merge( df, dfh, on = ['country','model'], how = 'left' )
df = pd.merge( df, dfc, on = ['country'], how = 'left').drop( columns = ['country'] )
df.year = df.year + 5

df = pd.concat( (df, df2) )

### annoying issue with 2095-2100 not being the same as the other decades ###
df2090 = df[df.year != 2100]
df2100 = df[df.year == 2100]
df2100b = df2100[df2100.model == 'MRI-ESM2-0']
df2100 = df2100[df2100.model != 'MRI-ESM2-0']
df2100.produ *= 1.67 # six years... except for MRI, which is the full decade

df = pd.concat( ( df2090, df2100, df2100b ) )
df['heat_incease'] = df.produ - df.pro_h
df = df.dropna()
df.head(5)

In [ ]:
df = pd.merge(df, dfL, on = ['ISO', 'year', 'ssp'])

df['risk'] = df.heat_incease * df.livestock
df['risk2020'] = df.heat_incease * df.livestock2020
df = df[ ['REGION','model','year','ssp','livestock','risk','risk2020'] ]

dfr = df.groupby( ['REGION','year','model','ssp'], as_index = False ).sum()

dfr = dfr.groupby( ['REGION','year','ssp'], as_index = False ).mean()
dfr

In [ ]:
27759871240/1e9

In [ ]:
colors = ['#3c78d8', '#6aa84f', '#e69138', '#a61c00','#674ea7']
colors = [rgb2hex(i) for i in px.colors.sequential.deep[1::3]]

plt.rcParams.update( {'font.size': 8, 'xtick.labelsize' : 6, 'ytick.labelsize' : 6, 'legend.fontsize': 6, 'legend.frameon': False, 
                     'axes.linewidth':0.4, 'xtick.major.width':0.4, 'ytick.major.width':0.4, 'xtick.major.size':1, 'ytick.major.size':1,
                     'xtick.bottom': True, 'xtick.top': False, 'ytick.right': False, 'ytick.right': False, 'legend.numpoints': 4,
                     'xtick.major.pad':2.5, 'ytick.major.pad':2.5, 'axes.labelpad':2,'xtick.direction': 'out', 'ytick.direction': 'out', 
                      'axes.edgecolor':'black','ytick.color':'black','xtick.color':'black','axes.facecolor':'white'} )

plt.close('all')
fig = plt.figure( figsize = ( 4.5, 2.25 ) )
gs = GridSpec(1, 2) # height_ratios = [1, 0.05], width_ratios = [0.4, 1, 0.05] )
gs.update(wspace = 0.1, hspace = 0.0)

newyears = np.arange(2020,2101,1)
k = 2

def make_subplot( ssp, yaxis = True ):
    factor = 3650e9
    dft = dfr[dfr.ssp == ssp]
    regions = ['R5.2OECD','R5.2ASIA', 'R5.2LAM', 'R5.2MAF']
    for i, region in enumerate(regions):
        dft1 = dft[dft.REGION == region]

        spl1 = make_interp_spline(dft1.year, dft1.risk/factor, k=k)
        spl2 = make_interp_spline(dft1.year, dft1.risk2020/factor, k=k)
        plt.plot(newyears, spl1(newyears),    lw = 1, color = colors[i], zorder = 3)
        plt.plot(newyears, spl2(newyears),  lw = 1, color = colors[i], linestyle = ':', zorder = 1)

        plt.scatter(dft1.year, dft1.risk/factor, edgecolors = colors[i], facecolors = 'none', s = 5, marker = 's', zorder = 5, lw =0.5)

    plt.xticks([2020,2040,2060,2080,2100], [2020,2040,2060,2080,2100])
    plt.ylim(0,6.7)
    plt.yticks([0,1,2,3,4,5,6], [0,1,2,3,4,5,6])
    if yaxis == False: 
        ax1.axes.get_yaxis().set_visible(False)
        plt.yticks([0], [])
    else:
        plt.yticks(rotation = 90, va = 'center')

    sns.despine( offset = 3, trim = True )

ax1 = plt.subplot( gs[0,0] )
make_subplot( 'SSP2-45')

ax1 = plt.subplot( gs[0,1] )
make_subplot( 'SSP3-70', False)

plt.savefig( '/content/drive/My Drive/figures/livestock/figure_4b.png', dpi = 500, bbox_inches = 'tight', pad_inches = 0.00 )

In [ ]:
quants = np.zeros((11))
quants[1:] = np.quantile(df.heat_incease.values, np.arange(0.1,1.1,0.1))

regions = ['R5.2OECD','R5.2ASIA', 'R5.2LAM', 'R5.2MAF']
colors = [rgb2hex(i) for i in px.colors.sequential.deep[1::3]]

plt.rcParams.update( {'font.size': 8, 'xtick.labelsize' : 6, 'ytick.labelsize' : 6, 'legend.fontsize': 6, 'legend.frameon': False, 
                     'axes.linewidth':0.4, 'xtick.major.width':0.4, 'ytick.major.width':0.4, 'xtick.major.size':1, 'ytick.major.size':1,
                     'xtick.bottom': True, 'xtick.top': False, 'ytick.right': False, 'ytick.right': False, 'legend.numpoints': 4,
                     'xtick.major.pad':2.5, 'ytick.major.pad':2.5, 'axes.labelpad':2,'xtick.direction': 'out', 'ytick.direction': 'out', 
                      'axes.edgecolor':'black','ytick.color':'black','xtick.color':'black','axes.facecolor':'white'} )

plt.close('all')
fig, ax = plt.subplots( figsize = (3.75, 3.75 ), frameon=True)
for i in range(10):
    dft = df[ df.heat_incease < quants[i+1] ]
    dft = dft[ dft.heat_incease > quants[i] ]    
    last_increase = 0
    for j, region in enumerate(regions):
        dft1 = dft[dft.REGION == region]
        increase = dft1.live_increase.values.sum()/1e6
        plt.bar(i, increase, bottom = last_increase, facecolor = colors[j], edgecolor='none', alpha = 0.9)
        last_increase += increase

ax.spines['bottom'].set_color('white')
#ax.tick_params(axis='x', colors='white')
ax.xaxis.label.set_color('black')

plt.xticks([0,1,2,3,4,5,6,7,8,9],[1,2,3,4,5,6,7,8,9,10] )
sns.despine( offset = 3, trim = True )
plt.yticks(rotation = 90, va = 'center')
plt.xlabel('Heat increase quantile')
plt.ylabel('Cattle increase (million head)')

plt.savefig( '/content/drive/My Drive/figures/livestock/figure_animals_per_heat_quantile.png', dpi = 500, bbox_inches = 'tight', pad_inches = 0.00 )

# Land USE

In [ ]:
### Process the LUH2 data to country level ###
ssp585 = '/content/drive/My Drive/data/livestock/land_use/multiple-states_input4MIPs_landState_ScenarioMIP_UofMD-MAGPIE-ssp585-2-1-f_gn_2015-2100.nc'
ssp370 = '/content/drive/My Drive/data/livestock/land_use/multiple-states_input4MIPs_landState_ScenarioMIP_UofMD-AIM-ssp370-2-1-f_gn_2015-2100.nc'
ssp126 = '/content/drive/My Drive/data/livestock/land_use/multiple-states_input4MIPs_landState_ScenarioMIP_UofMD-IMAGE-ssp126-2-1-f_gn_2015-2100.nc'

mask = xr.load_dataarray('/content/drive/Shared drives/ClimateAg/data/gadm36_level0_0_25_180_180.nc4')
mask = np.flip( np.nan_to_num( mask.values ), axis = 0 )

x = xr.open_dataset(ssp126, decode_times=False)
pastr = np.nan_to_num( x.pastr.values ) * cellarea
rang  = np.nan_to_num( x.range.values ) * cellarea
c3  = np.nan_to_num( x.c3ann.values )   * cellarea
c4  = np.nan_to_num( x.c4ann.values )   * cellarea
c3n  = np.nan_to_num( x.c3nfx.values )  * cellarea
crop = c3 + c4 + c3n

years = np.arange( 2015, 2100, 1 )
df = []
for i in range( 85 ):
    print(i)
    for j in np.unique( mask ):
        df.append({'year':  years[i], 'country': j, 
                   'pastr': np.ma.masked_where(mask != j, pastr[i,:,:]).sum(),
                   'rang':  np.ma.masked_where(mask != j, rang[i,:,:]).sum(),
                   'crop':  np.ma.masked_where(mask != j, crop[i,:,:]).sum()})

df = pd.DataFrame(df)
df.to_csv('/content/drive/My Drive/data/livestock/land_use/ssp126_country_level.csv', index = False )

In [ ]:
df.head(5)

In [ ]:
### get unique list of countries from FAO and GADM countries... need to rename some GADM countries to FAO designation 
df = pd.read_csv( '/content/drive/My Drive/data/livestock/country_level_data/Production_Livestock_E_All_Data_NOFLAG.csv', sep = ',', encoding = 'iso-8859-1' )
df = df[['Area','Item']]
df = df.rename( columns = {'Area':'countryname'} )
df = df[df.Item == 'Cattle']
GADM =  pd.read_csv( '/content/drive/My Drive/data/area_masks/gadm36_country_names_and_abbr.csv', sep = ',' )
GADM = GADM.rename( columns = {'NAME_0':'countryname', 'GID_0': 'ISO'} )
GADM['maskid'] = GADM.index
GADM = GADM[['ISO','maskid']]
FAO =  pd.read_csv( '/content/drive/My Drive/data/area_masks/countrycodes_FAO_NAMES.csv', sep = ',' ).rename(columns = {'abb':'ISO'})
FAO = FAO[['ISO','countryname']]
countrynames = pd.merge( FAO, GADM, on = [ 'ISO' ] , how = 'inner' )

smallisland = ['Barbados','Bermuda' ,'Dominica','Bahama','Grenada','Guam','Montserrat','Malta','Nauru','Niue','Saint Kitts and Nevis', 
               'Saint Pierre and Miquelon', 'St. Vincent and the Grenadines','Singapore','Tonga','Tuvalu','Wallis and Futuna Islands']
df = df[ ~df.countryname.isin(smallisland) ]
dfc = pd.merge( df, countrynames, on = [ 'countryname' ] , how = 'inner' )
countrynames = dfc.maskid.unique()

dfc = dfc.rename( columns = {'maskid': 'country'} ).drop( columns = ['Item'] )

regions = pd.read_excel('/content/drive/My Drive/data/econ/ssps/cmip6_iam_model_region_mapping.xlsx')
regions = regions[ ['ISO', 'R5_region'] ]
regions = regions.replace( ['R5LAM', 'R5MAF', 'R5REF', 'R5OECD', 'R5ASIA'], ['R5.2LAM', 'R5.2MAF', 'R5.2REF', 'R5.2OECD', 'R5.2ASIA'] )

In [ ]:
df =  pd.read_csv( '/content/drive/My Drive/data/livestock/land_use/ssp370_country_level.csv' )
df['land'] = df.pastr + df.rang
dfh = df[df.year == 2015]
dfh = dfh.drop( columns = ['year','pastr','rang']).rename(columns = {'land':'land_h'})
df = pd.merge( df, dfh, on = 'country')
df['per_change'] = (df.land - df.land_h) / df.land_h
df = pd.merge( df, dfc, on = 'country' )
df = pd.merge( df, regions, on = 'ISO' )
df = df[df.R5_region == 'R5.2MAF']
dft = df[df.year == 2015]
dft.land.sum()

In [ ]:
(26245777.043780785 - 21738699.800947774)/ 21738699.800947774

In [ ]:
dft = df[df.year == 2099]
dft.land.sum()

In [ ]:
1/30

In [ ]:
colors = [rgb2hex(i) for i in px.colors.sequential.deep[1::3]]

plt.rcParams.update( {'font.size': 8, 'xtick.labelsize' : 6, 'ytick.labelsize' : 6, 'legend.fontsize': 6, 'legend.frameon': False, 
                     'axes.linewidth':0.4, 'xtick.major.width':0.4, 'ytick.major.width':0.4, 'xtick.major.size':1, 'ytick.major.size':1,
                     'xtick.bottom': True, 'xtick.top': False, 'ytick.right': False, 'ytick.right': False, 'legend.numpoints': 4,
                     'xtick.major.pad':2.5, 'ytick.major.pad':2.5, 'axes.labelpad':2,'xtick.direction': 'out', 'ytick.direction': 'out', 
                      'axes.edgecolor':'black','ytick.color':'black','xtick.color':'black','axes.facecolor':'white'} )

plt.close('all')
fig = plt.figure( figsize = ( 5, 5 ) )
for country in df.country.unique():
    dft = df[df.country == country]
    if dft.R5_region.unique() == 'R5.2OECD': color = colors[0]
    elif dft.R5_region.unique() == 'R5.2ASIA': color = colors[1]
    elif dft.R5_region.unique() == 'R5.2LAM': color = colors[2]
    elif dft.R5_region.unique() == 'R5.2MAF': color = colors[3]
    else: color = 'black'
    plt.plot(dft.year, dft.per_change, color = color, lw = 0.8, alpha = 0.8)

dft = df[df.year == 2099]
dft = dft.sort_values(by='land', ascending=False)
dft = dft[dft.per_change < 3]

for index, row in dft.iterrows():
    plt.annotate(row.countryname, xy = (2100, row.per_change), fontsize = 5)

plt.xlim(2015, 2130)
plt.ylim(-1, 1)
plt.xlabel('Year')
plt.ylabel('Total pasture and rangland (ha)')

sns.despine( offset = 3, trim = True )
plt.yticks(rotation = 90, va = 'center')

plt.savefig( '/content/drive/My Drive/figures/livestock/land_change_ssp370.png', dpi = 500, bbox_inches = 'tight', pad_inches = 0.00 )

In [ ]:
'Democratic Republic of the Congo', 
'Sudan', 
'Saudi Arabia',
'Angola', 
'Chad', 
'Mozambique', 
'South Africa', 
'Ethiopia',
'South Sudan', 
'United Republic of Tanzania',

In [ ]:
df = df[df.year == 2099]
df = df.sort_values(by='land', ascending=False)
df.countryname.values



---

# OLD

---



In [ ]:
mask = xr.load_dataarray('/content/drive/My Drive/data/area_masks/gadm36_countries_0.25_degrees.nc4')
mask = cv2.resize(mask.values, (1440,720))
mask = np.flip(mask, axis = 0 )
new = np.zeros_like(mask)
new[:, 720:] = mask[:, :720]
new[:, :720] = mask[:, 720:]
plt.pcolormesh(new)

In [ ]:
mask = xr.load_dataarray('/content/drive/My Drive/data/area_masks/gadm36_countries_0.25_degrees.nc4')
mask = cv2.resize(mask.values, (1440,720))
mask = np.flip(mask, axis = 0 )
new = np.zeros_like(mask)
new[:, 720:] = mask[:, :720]
new[:, :720] = mask[:, 720:]

In [ ]:
df = pd.read_csv( 'https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv' )

df1 = df.query( "source_id == 'HadGEM2-ES'" )
df1

In [ ]:
df = pd.read_csv( 'https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv' )

df1 = df.query( "activity_id=='ScenarioMIP' & variable_id == 'tas' & experiment_id == 'ssp126' & table_id == 'day'" )
tas = df1.source_id.unique().tolist()
df1 = df.query( "activity_id=='ScenarioMIP' & variable_id == 'huss' & experiment_id == 'ssp126' & table_id == 'day'" )
huss = df1.source_id.unique().tolist()
ssp585_models = list( set(tas).intersection( set(huss) ) )

for model in ssp585_models:
    df1 = df.query( f"activity_id=='ScenarioMIP' & variable_id == 'tas' & experiment_id == 'ssp126' & table_id == 'day' & source_id == '{model}'" )
    Ta = xr.open_zarr( gcs.get_mapper( df1.zstore.values[0] ), consolidated = True )
    print(model, Ta.time.values[-1])

In [ ]:
era = xr.open_dataarray('/content/drive/My Drive/data/livestock/ERA5/era_doy_media.nc4')
lon = era.lon.values
lat = era.lat.values
doy = era.dayofyear.values
pro_thresh = 67.9     # production threshold
dea_thresh = 73       # death and fertility threshold
era_thresh = np.ma.masked_where( era< pro_thresh, era )
era_pro = np.ma.count( era_thresh, axis = 0 )
era_thresh = np.ma.masked_where( era< dea_thresh, era )
era_dea = np.ma.count( era_thresh, axis = 0 )
era_pro_HL = np.sum( np.ma.masked_where( era < pro_thresh, era ) - pro_thresh, axis = 0 )
era_pro, lons = shiftgrid(180., era_pro, lon, start=False)
era_dea, lons = shiftgrid(180., era_dea, lon, start=False)
era_pro_HL, lons = shiftgrid(180., era_pro_HL, lon, start=False)

lon2d, lat2d = np.meshgrid( lons, lat )

era_pro = maskoceans( lon2d, lat2d, era_pro )
era_pro_HL = maskoceans( lon2d, lat2d, era_pro_HL )

In [ ]:
era = xr.open_dataarray('/content/drive/My Drive/data/livestock/ERA5/era_doy_media.nc4')
lon = era.lon.values
lat = era.lat.values
doy = era.dayofyear.values
pro_thresh = 67.9     # production threshold
dea_thresh = 73       # death and fertility threshold
era_thresh = np.ma.masked_where( era< pro_thresh, era )
era_pro = np.ma.count( era_thresh, axis = 0 )
era_thresh = np.ma.masked_where( era< dea_thresh, era )
era_dea = np.ma.count( era_thresh, axis = 0 )
era_pro_HL = np.sum( np.ma.masked_where( era < pro_thresh, era ) - pro_thresh, axis = 0 )
era_pro, lons = shiftgrid(180., era_pro, lon, start=False)
era_dea, lons = shiftgrid(180., era_dea, lon, start=False)
era_pro_HL, lons = shiftgrid(180., era_pro_HL, lon, start=False)

lon2d, lat2d = np.meshgrid( lons, lat )

era_pro = maskoceans( lon2d, lat2d, era_pro )
era_pro_HL = maskoceans( lon2d, lat2d, era_pro_HL )

root = '/content/drive/My Drive/data/livestock/CMIP6/'
ssp126 = [root + i for i in fnmatch.filter( os.listdir(root), '*ssp126*68_73*.nc4')]
ssp585 = [root + i for i in fnmatch.filter( os.listdir(root), '*ssp585*68_73*.nc4')]

ssp5 = np.ma.zeros((9,721,1440))
for i, file in enumerate(ssp585):
    temp = xr.open_dataset(file)
    temp = temp.pro_days
    temp, lons = shiftgrid(180., temp, lon, start=False)
    ssp5[i,:,:] = maskoceans( lon2d, lat2d, temp )

ssp1 = np.ma.zeros((9,721,1440))
for i, file in enumerate(ssp126):
    temp = xr.open_dataset(file)
    temp = temp.pro_days
    temp, lons = shiftgrid(180., temp, lon, start=False)
    ssp1[i,:,:] = maskoceans( lon2d, lat2d, temp )

ssp1m = np.median( ssp1, axis = 0 )
ssp5m = np.median( ssp5, axis = 0 )

ssp1m = maskoceans( lon2d, lat2d, ssp1m )
ssp5m = maskoceans( lon2d, lat2d, ssp5m )

ssp5 = np.ma.zeros((9,721,1440))
for i, file in enumerate(ssp585):
    temp = xr.open_dataset(file)
    temp = temp.pro_HL
    temp, lons = shiftgrid(180., temp, lon, start=False)
    ssp5[i,:,:] = maskoceans( lon2d, lat2d, temp )

ssp1 = np.ma.zeros((9,721,1440))
for i, file in enumerate(ssp126):
    temp = xr.open_dataset(file)
    temp = temp.pro_HL
    temp, lons = shiftgrid(180., temp, lon, start=False)
    ssp1[i,:,:] = maskoceans( lon2d, lat2d, temp )

ssp1mh = np.median( ssp1, axis = 0 )
ssp5mh = np.median( ssp5, axis = 0 )

ssp1mh = maskoceans( lon2d, lat2d, ssp1mh )
ssp5mh = maskoceans( lon2d, lat2d, ssp5mh )

# Livestock numbers based on IAM projections to 2100

In [ ]:
### SSP projections from IIASA ###
iam = pd.read_csv('/content/drive/My Drive/data/econ/ssps/SSP_IAM_V2_201811.csv')
iam = iam[iam.VARIABLE == 'Agricultural Demand|Livestock']
iam = iam.melt( id_vars = ['MODEL', 'SCENARIO', 'REGION', 'VARIABLE', 'UNIT'], var_name = 'year', value_name = 'value' )
iam = iam.groupby( by = ['SCENARIO', 'year', 'REGION'], as_index = False ).mean()
iam.year = pd.to_numeric( iam.year )

### get percent change from 2010 baseline ###
iam2010 = iam[ iam.year == 2010 ]
iam2010 = iam2010.drop( columns = ('year') ).rename( columns = {'value':'baseline'} )
iam = pd.merge( iam, iam2010, on = ['SCENARIO', 'REGION'] )
iam['change'] = (iam.value - iam.baseline) / iam.baseline * 100
iam['ssp']  = iam.SCENARIO.apply( lambda x: x.split('-')[0] )
iam = iam.drop( columns = ['value', 'baseline'] )

### region designations from FAO ###
regions = pd.read_excel('/content/drive/My Drive/data/econ/ssps/cmip6_iam_model_region_mapping.xlsx')
regions = regions[ ['ISO', 'Country', 'R5_region'] ]
regions= regions.replace( ['R5LAM', 'R5MAF', 'R5REF', 'R5OECD', 'R5ASIA'], ['R5.2LAM', 'R5.2MAF', 'R5.2REF', 'R5.2OECD', 'R5.2ASIA'] )

### livestock numbers from FAO ###
df = pd.read_csv( '/content/drive/My Drive/data/livestock/Environment_LivestockPatterns_E_All_Data_NOFLAG.csv', sep = ',', encoding = 'iso-8859-1' )
df = df.drop( columns = ['Area Code', 'Element Code', 'Item Code'] )
df = df.melt( id_vars = ['Area', 'Element', 'Item', 'Unit'], var_name = 'year', value_name = 'value' ).rename( columns = { 'Area' : 'Country' } )
df.year = df.year.apply( lambda x: x.split('Y')[1] ).astype( int )
df = df[df.Element == 'Stocks']
df = df[df.Item == 'Cattle']
df = df[df.Country != 'World']
df = pd.merge( df, regions, on = ['Country'] ).rename( columns = {'R5_region':'REGION'})
df = df[df.year==2010]
df = df.drop( columns = ('year') )

### Combine country level livestock data with regional ssp projected changes ###
df = pd.merge(df, iam, on = ['REGION'])
df['livestock'] = df.value * (1 + df.change / 100)
dfl = df[ ['Country','ISO','REGION','SCENARIO','year','livestock'] ]
dfl = dfl.rename( columns = { 'SCENARIO': 'ssp' } )
dfb = dfl[ dfl.year == 2020 ]
dfb = dfb.rename(columns={'livestock':'livestock2020'}).drop(columns = 'year')
dfl = pd.merge( dfl, dfb, on = ['Country','ISO','REGION','ssp'] )
dfl = dfl.replace( ['SSP5-Baseline', 'SSP3-Baseline'], ['SSP5-85', 'SSP3-70'] )
dfl.head(20)

In [ ]:
### get unique list of countries from FAO and GADM countries... need to rename some GADM countries to FAO designation 
df = pd.read_csv( '/content/drive/My Drive/data/livestock/country_level_data/Production_Livestock_E_All_Data_NOFLAG.csv', sep = ',', encoding = 'iso-8859-1' )
df = df[['Area','Item']]
df = df.rename( columns = {'Area':'countryname'} )
df = df[df.Item == 'Cattle']
GADM =  pd.read_csv( '/content/drive/My Drive/data/area_masks/gadm36_country_names_and_abbr.csv', sep = ',' )
GADM = GADM.rename( columns = {'NAME_0':'countryname', 'GID_0': 'ISO'} )
GADM['maskid'] = GADM.index
GADM = GADM[['ISO','maskid']]
FAO =  pd.read_csv( '/content/drive/My Drive/data/area_masks/countrycodes_FAO_NAMES.csv', sep = ',' ).rename(columns = {'abb':'ISO'})
FAO = FAO[['ISO','countryname']]
countrynames = pd.merge( FAO, GADM, on = [ 'ISO' ] , how = 'inner' )

smallisland = ['Barbados','Bermuda' ,'Dominica','Bahama','Grenada','Guam','Montserrat','Malta','Nauru','Niue','Saint Kitts and Nevis', 
               'Saint Pierre and Miquelon', 'St. Vincent and the Grenadines','Singapore','Tonga','Tuvalu','Wallis and Futuna Islands']
df = df[ ~df.countryname.isin(smallisland) ]
dfc = pd.merge( df, countrynames, on = [ 'countryname' ] , how = 'inner' )
countrynames = dfc.maskid.unique()

dfc = dfc.rename( columns = {'maskid': 'country'} ).drop( columns = [ 'countryname',  'Item'] )

countrymaskgrid  = xr.open_dataarray( '/content/drive/My Drive/data/area_masks/gadm36_countries_FOA_animal_dist_resolution.nc4' ).values

plt.pcolormesh( countrymaskgrid )

# Combine T and Animal projections

In [ ]:
df = pd.read_csv( '/content/drive/My Drive/data/livestock/vitali_all_rcp_country.csv' )
df = df.replace( { 'ssp126': 'SSP1-26', 'ssp119':'SSP1-19', 'ssp245': 'SSP2-45', 'ssp370':'SSP3-70', 'ssp585':'SSP5-85'} )
df = pd.merge( df, dfc, on = 'country' )
df = pd.merge( df, dfl, on = ['ISO','year','ssp'] )
df = df[df.risk != '--']
df.risk = pd.to_numeric(df.risk)
df = df.dropna()
df['deaths'] = df.livestock * df.risk * 232.75 / 1867397.0 ## from Vitali 2009
df['deaths2020'] = df.livestock2020 * df.risk * 232.75 / 1867397.0 ## from Vitali 2009
df.head(10)

In [ ]:
df = pd.read_csv( '/content/drive/My Drive/data/livestock/vitali_all_rcp_country.csv' )
df = df.replace( { 'ssp126': 'SSP1-26', 'ssp119':'SSP1-19', 'ssp245': 'SSP2-45', 'ssp370':'SSP3-70', 'ssp585':'SSP5-85'} )
df

In [ ]:
df = df.groupby( ['model','ssp','year', 'REGION'], as_index = False ).sum()
df = df.groupby( ['ssp','year', 'REGION'], as_index = False ).mean()
df.head(10)

In [ ]:
colors = ['#1b9e77','#d95f02','#7570b3','#e7298a','#66a61e']
regions = ['R5.2ASIA', 'R5.2LAM', 'R5.2MAF', 'R5.2OECD']
ssps = df.ssp.unique()
plt.figure(figsize=(15,5))
gs  = gridspec.GridSpec( 1, 4 )
gs.update( wspace = 0.05, hspace = 0.0)

ax1 = plt.subplot( gs[0, 0] )
df1 = df[df.REGION == 'R5.2ASIA']
for i, ssp in enumerate(ssps):
    dfr = df1[df1.ssp == ssp]
    plt.plot( dfr.year, dfr.livestock, color = colors[i], label = ssp)
    plt.scatter( dfr.year, dfr.livestock, edgecolor = colors[i], facecolor = 'none' )

plt.ylim(0,8.5e8)
plt.legend(frameon = False)
plt.title('Asia')

ax1 = plt.subplot( gs[0, 1] )
df1 = df[df.REGION == 'R5.2LAM']
for i, ssp in enumerate(ssps):
    dfr = df1[df1.ssp == ssp]
    plt.plot( dfr.year, dfr.livestock, color = colors[i], label = region)
    plt.scatter( dfr.year, dfr.livestock, edgecolor = colors[i], facecolor = 'none' )

plt.yticks([1e8, 2e8, 3e8, 4e8, 5e8, 6e8, 7e8, 8e8],[])
plt.ylim(0,8.5e8)
plt.title('Latin America')

ax1 = plt.subplot( gs[0, 2] )
df1 = df[df.REGION == 'R5.2MAF']
for i, ssp in enumerate(ssps):
    dfr = df1[df1.ssp == ssp]
    plt.plot( dfr.year, dfr.livestock, color = colors[i], label = region)
    plt.scatter( dfr.year, dfr.livestock, edgecolor = colors[i], facecolor = 'none' )

plt.yticks([1e8, 2e8, 3e8, 4e8, 5e8, 6e8, 7e8, 8e8],[])
plt.ylim(0,8.5e8)
plt.title('Africa')

ax1 = plt.subplot( gs[0, 3] )
df1 = df[df.REGION == 'R5.2OECD']
for i, ssp in enumerate(ssps):
    dfr = df1[df1.ssp == ssp]
    plt.plot( dfr.year, dfr.livestock, color = colors[i], label = region)
    plt.scatter( dfr.year, dfr.livestock, edgecolor = colors[i], facecolor = 'none' )

plt.yticks([1e8, 2e8, 3e8, 4e8, 5e8, 6e8, 7e8, 8e8],[])
plt.ylim(0,8.5e8)
plt.title('OECD')
plt.savefig( '/content/drive/My Drive/figures/cow_projections.png', dpi = 500, bbox_inches = 'tight', pad_inches = 0.01 )

In [ ]:
colors = ['#e41a1c','#377eb8','#4daf4a','#984ea3','#ff7f00']
regions = ['R5.2ASIA', 'R5.2LAM', 'R5.2MAF', 'R5.2OECD']

plt.figure(figsize=(15,5))
gs  = gridspec.GridSpec( 1, 4 )
gs.update( wspace = 0.05, hspace = 0.0)

ax1 = plt.subplot( gs[0, 0] )
df1 = df[df.ssp == 'SSP1-19']
for i, region in enumerate(regions):
    dfr = df1[df1.REGION == region]
    plt.plot( dfr.year, dfr.deaths, color = colors[i], label = region)
    plt.scatter( dfr.year, dfr.deaths, edgecolor = colors[i], facecolor = 'none' )
    plt.plot( dfr.year, dfr.deaths2020, color = colors[i], linestyle = '--' )

plt.ylim(0,7e7)
plt.legend(frameon = False)
plt.title('Agressive Mitigation')

ax1 = plt.subplot( gs[0, 1] )
df1 = df[df.ssp == 'SSP2-45']
for i, region in enumerate(regions):
    dfr = df1[df1.REGION == region]
    plt.plot( dfr.year, dfr.deaths, color = colors[i], label = region)
    plt.scatter( dfr.year, dfr.deaths, edgecolor = colors[i], facecolor = 'none' )
    plt.plot( dfr.year, dfr.deaths2020, color = colors[i], linestyle = '--' )
plt.yticks([1e7, 2e7, 3e7, 4e7, 5e7, 6e7, 7e7],[])
plt.ylim(0,7e7)
plt.title('Mitigation')

ax1 = plt.subplot( gs[0, 2] )
df1 = df[df.ssp == 'SSP3-70']
for i, region in enumerate(regions):
    dfr = df1[df1.REGION == region]
    plt.plot( dfr.year, dfr.deaths, color = colors[i], label = region)
    plt.scatter( dfr.year, dfr.deaths, edgecolor = colors[i], facecolor = 'none' )
    plt.plot( dfr.year, dfr.deaths2020, color = colors[i], linestyle = '--' )
plt.yticks([1e7, 2e7, 3e7, 4e7, 5e7, 6e7, 7e7],[])
plt.ylim(0,7e7)
plt.title('Regional Rivalry')

ax1 = plt.subplot( gs[0, 3] )
df1 = df[df.ssp == 'SSP5-85']
for i, region in enumerate(regions):
    dfr = df1[df1.REGION == region]
    plt.plot( dfr.year, dfr.deaths, color = colors[i], label = region)
    plt.scatter( dfr.year, dfr.deaths, edgecolor = colors[i], facecolor = 'none' )
    plt.plot( dfr.year, dfr.deaths2020, color = colors[i], linestyle = '--' )
plt.yticks([1e7, 2e7, 3e7, 4e7, 5e7, 6e7, 7e7],[])
plt.ylim(0,7e7)
plt.title('Business as usuall')
plt.savefig( '/content/drive/My Drive/figures/3rcp_cow_projections.png', dpi = 500, bbox_inches = 'tight', pad_inches = 0.01 )

In [ ]:
df1 = df[df.ssp == 'SSP1-19']
regions = df1.REGION.unique()
colors = ['#e41a1c','#377eb8','#4daf4a','#984ea3','#ff7f00']
regions = ['R5.2ASIA', 'R5.2LAM', 'R5.2MAF', 'R5.2OECD']
plt.figure(figsize=(10,8))
for i, region in enumerate(regions):
    dfr = df1[df1.REGION == region]
    plt.plot( dfr.year, dfr.deaths, color = colors[i], label = region)
    plt.scatter( dfr.year, dfr.deaths, edgecolor = colors[i], facecolor = 'none' )
    plt.plot( dfr.year, dfr.deaths2020, color = colors[i], linestyle = '--' )

plt.legend(frameon = False)